In [1]:
import pygame
import sys
import heapq

# ขนาดหน้าจอ
WIDTH, HEIGHT = 800, 600
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
ROBOT_COLOR = (0, 128, 255)
TARGET_COLOR = (255, 0, 0)
WALL_COLOR = (100, 100, 100)

# กำหนดขนาดกริด (cell size)
GRID_SIZE = 20

# พิกัดเป้าหมาย
TARGET_POSITIONS = {
    pygame.K_1: (150, 500),
    pygame.K_2: (200, 100),
    pygame.K_3: (650, 300)
}

# กำแพง 4 อัน แบบง่าย ๆ
walls = [
    pygame.Rect(200, 150, 200, 20),  # กำแพงแนวนอนบน1
    pygame.Rect(500, 150, 200, 20),  # กำแพงแนวนอนบน2
    pygame.Rect(200, 450, 200, 20),  # กำแพงแนวนอนล่าง
    pygame.Rect(600, 450, 150, 20),  # กำแพงแนวนอนล่าง
    pygame.Rect(200, 150, 20, 300),  # กำแพงแนวตั้งซ้าย
    pygame.Rect(580, 50, 20, 500),   # กำแพงแนวตั้งขวา
    pygame.Rect(50, 300, 150, 20),  # กำแพงแนวนอนกลาง
]

# เริ่มต้นตำแหน่งหุ่นยนต์ (pixel)
robot_pos = [400, 300]
target_pos = None
robot_speed = 2.5

# ตัวแปรเก็บเส้นทางที่หาได้ (เป็น list ของตำแหน่ง grid)
path = []

# เริ่ม pygame
pygame.init()
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Robot A* Pathfinding")
clock = pygame.time.Clock()
font = pygame.font.SysFont(None, 36)


def draw_text(text, pos):
    img = font.render(text, True, BLACK)
    screen.blit(img, pos)


def is_blocked(x, y):
    """ตรวจสอบว่า grid cell (x,y) ชนกำแพงหรือไม่"""
    rect = pygame.Rect(x * GRID_SIZE, y * GRID_SIZE, GRID_SIZE, GRID_SIZE)
    return any(rect.colliderect(wall) for wall in walls)


def heuristic(a, b):
    # Octile distance heuristic สำหรับเดินทะแยง
    dx = abs(a[0] - b[0])
    dy = abs(a[1] - b[1])
    D = 1
    D2 = 1.414  # sqrt(2)
    return D * (dx + dy) + (D2 - 2 * D) * min(dx, dy)


def a_star_search(start, goal):
    """ฟังก์ชันค้นหาเส้นทางด้วย A* แบบ 8 ทิศทาง"""
    neighbors = [
        (0, 1), (1, 0), (0, -1), (-1, 0),   # 4 ทิศหลัก
        (1, 1), (1, -1), (-1, 1), (-1, -1)  # 4 ทิศเฉียง
    ]

    close_set = set()
    came_from = {}
    gscore = {start: 0}
    fscore = {start: heuristic(start, goal)}
    oheap = []

    heapq.heappush(oheap, (fscore[start], start))

    while oheap:
        current = heapq.heappop(oheap)[1]

        if current == goal:
            data = []
            while current in came_from:
                data.append(current)
                current = came_from[current]
            data.reverse()
            return data

        close_set.add(current)
        for i, j in neighbors:
            neighbor = current[0] + i, current[1] + j
            if neighbor[0] < 0 or neighbor[1] < 0 or neighbor[0] >= WIDTH // GRID_SIZE or neighbor[1] >= HEIGHT // GRID_SIZE:
                continue
            if is_blocked(neighbor[0], neighbor[1]):
                continue

            # คำนวณค่า tentative_g_score สำหรับเดินทิศทางเฉียงและแนวตรง
            if abs(i) + abs(j) == 2:
                tentative_g_score = gscore[current] + 1.414
            else:
                tentative_g_score = gscore[current] + 1

            if neighbor in close_set and tentative_g_score >= gscore.get(neighbor, 0):
                continue

            if tentative_g_score < gscore.get(neighbor, float('inf')) or neighbor not in [i[1] for i in oheap]:
                came_from[neighbor] = current
                gscore[neighbor] = tentative_g_score
                fscore[neighbor] = tentative_g_score + heuristic(neighbor, goal)
                heapq.heappush(oheap, (fscore[neighbor], neighbor))
    return []


def move_robot_along_path():
    global path, robot_pos
    if path:
        target_cell = path[0]
        target_x = target_cell[0] * GRID_SIZE + GRID_SIZE // 2
        target_y = target_cell[1] * GRID_SIZE + GRID_SIZE // 2

        dx = target_x - robot_pos[0]
        dy = target_y - robot_pos[1]
        dist = (dx ** 2 + dy ** 2) ** 0.5

        if dist < robot_speed:
            # ไปถึง cell นี้แล้ว ลบออกจาก path
            path.pop(0)
        else:
            robot_pos[0] += robot_speed * dx / dist
            robot_pos[1] += robot_speed * dy / dist


while True:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()
        elif event.type == pygame.KEYDOWN:
            if event.key in TARGET_POSITIONS:
                target_pos = TARGET_POSITIONS[event.key]

                # แปลงตำแหน่ง robot และ target เป็น grid
                start_grid = (int(robot_pos[0] // GRID_SIZE), int(robot_pos[1] // GRID_SIZE))
                goal_grid = (int(target_pos[0] // GRID_SIZE), int(target_pos[1] // GRID_SIZE))

                # หา path ด้วย A*
                path = a_star_search(start_grid, goal_grid)

    screen.fill(WHITE)

    # วาดกำแพง
    for wall in walls:
        pygame.draw.rect(screen, WALL_COLOR, wall)

    # วาดเป้าหมาย
    for idx, pos in enumerate(TARGET_POSITIONS.values(), start=1):
        pygame.draw.circle(screen, TARGET_COLOR, pos, 15)
        draw_text(f'{idx}', (pos[0] - 10, pos[1] - 40))

    # เคลื่อนที่หุ่นยนต์ตาม path
    move_robot_along_path()

    # วาดเส้นทาง (เส้นสีฟ้า)
    if path:
        for cell in path:
            rect = pygame.Rect(cell[0] * GRID_SIZE, cell[1] * GRID_SIZE, GRID_SIZE, GRID_SIZE)
            pygame.draw.rect(screen, (0, 200, 255), rect)

    # วาดหุ่นยนต์
    pygame.draw.circle(screen, ROBOT_COLOR, (int(robot_pos[0]), int(robot_pos[1])), 20)

    draw_text("Press 1, 2, or 3 to move robot", (20, 20))
    pygame.display.flip()
    clock.tick(60)





pygame 2.6.1 (SDL 2.28.4, Python 3.10.16)
Hello from the pygame community. https://www.pygame.org/contribute.html


SystemExit: 

c:\Users\earth\anaconda3\envs\Car\lib\site-packages\IPython\core\interactiveshell.py:3587: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [1]:
import pygame
import sys
import heapq
import math
import random
from collections import deque

# --- พื้นฐาน ---
WIDTH, HEIGHT = 800, 600
WHITE, BLACK = (255, 255, 255), (0, 0, 0)
ROBOT_COLOR = (0, 128, 255)
TARGET_COLOR = (255, 0, 0)
WALL_COLOR = (100, 100, 100)
UNEXPLORED_COLOR = (50, 50, 50)  # สีดำสำหรับพื้นที่ที่ยังไม่ได้แสกน
EXPLORED_COLOR = (200, 200, 200)  # สีเทาอ่อนสำหรับพื้นที่ที่แสกนแล้ว
PATH_COLOR = (0, 200, 255)
GRID_SIZE = 20
ROBOT_RADIUS = 15

# --- กำแพง + เป้าหมาย ---
walls = [
    pygame.Rect(200, 150, 200, 20),
    pygame.Rect(500, 150, 200, 20),
    pygame.Rect(200, 450, 200, 20),
    pygame.Rect(600, 450, 150, 20),
    pygame.Rect(200, 150, 20, 300),
    pygame.Rect(580, 50, 20, 500),
    pygame.Rect(50, 300, 150, 20),
]

TARGET_POSITIONS = [
    (150, 500),
    (200, 100),
    (650, 300)
]

# --- ตัวแปรสถานะ ---
robot_pos = [100, 50]  # เริ่มต้นที่มุมซ้ายบน
robot_speed = 3
robot_angle = 0
path = []
current_target = None
targets_found = []
exploring = False
exploration_path = []

# --- การแสกนพื้นที่ ---
FOV_RADIUS = 80
FOV_ANGLE = 90
explored_cells = set()  # เซ็ตของเซลล์ที่แสกนแล้ว
wall_cells = set()      # เซ็ตของเซลล์ที่เป็นกำแพง

def get_grid_pos(world_pos):
    """แปลงพิกัดโลกเป็นพิกัดกริด"""
    return (int(world_pos[0] // GRID_SIZE), int(world_pos[1] // GRID_SIZE))

def get_world_pos(grid_pos):
    """แปลงพิกัดกริดเป็นพิกัดโลก"""
    return (grid_pos[0] * GRID_SIZE + GRID_SIZE // 2, grid_pos[1] * GRID_SIZE + GRID_SIZE // 2)

def is_wall_at(grid_pos):
    """ตรวจสอบว่าตำแหน่งกริดนั้นเป็นกำแพงหรือไม่"""
    world_pos = get_world_pos(grid_pos)
    test_rect = pygame.Rect(world_pos[0] - GRID_SIZE//2, world_pos[1] - GRID_SIZE//2, GRID_SIZE, GRID_SIZE)
    return any(test_rect.colliderect(wall) for wall in walls)

def scan_area(robot_pos, robot_angle):
    """แสกนพื้นที่รอบๆ หุ่นยนต์ตาม FOV"""
    global explored_cells, wall_cells
    
    robot_grid = get_grid_pos(robot_pos)
    
    # แสกนในรัศมี FOV
    for angle_offset in range(-FOV_ANGLE//2, FOV_ANGLE//2 + 1, 10):
        current_angle = math.radians(robot_angle + angle_offset)
        
        # ส่องไปในทิศทางนั้นจนถึงขอบเขต FOV
        for distance in range(0, FOV_RADIUS, GRID_SIZE//2):
            scan_x = robot_pos[0] + distance * math.cos(current_angle)
            scan_y = robot_pos[1] + distance * math.sin(current_angle)
            
            if not (0 <= scan_x < WIDTH and 0 <= scan_y < HEIGHT):
                break
                
            scan_grid = get_grid_pos((scan_x, scan_y))
            explored_cells.add(scan_grid)
            
            # ถ้าเจอกำแพงให้หยุด
            if is_wall_at(scan_grid):
                wall_cells.add(scan_grid)
                break

def find_nearest_safe_unexplored():
    """Find nearest unexplored area that's safely reachable"""
    robot_grid = get_grid_pos(robot_pos)
    candidates = []
    
    # Find border cells of explored areas
    for explored in explored_cells:
        for dx, dy in [(0,1), (1,0), (0,-1), (-1,0)]:
            neighbor = (explored[0] + dx, explored[1] + dy)
            
            # Check if it's within map bounds and unexplored
            if (0 <= neighbor[0] < WIDTH//GRID_SIZE and 
                0 <= neighbor[1] < HEIGHT//GRID_SIZE and
                neighbor not in explored_cells and
                neighbor not in wall_cells):
                
                # Test if we can reach this cell safely
                test_path = a_star_pathfind(robot_grid, neighbor, wall_cells, explored_cells)
                if test_path:  # If reachable
                    dist = len(test_path)
                    candidates.append((dist, neighbor))
    
    # Return the nearest reachable unexplored cell
    if candidates:
        candidates.sort()
        return candidates[0][1]
    
    return None

def check_for_targets():
    """Check for targets in scanned areas"""
    global targets_found
    
    for i, target_pos in enumerate(TARGET_POSITIONS):
        if i not in targets_found:
            target_grid = get_grid_pos(target_pos)
            if target_grid in explored_cells:
                targets_found.append(i)
                print(f"Target {i+1} found!")

def a_star_pathfind(start_grid, goal_grid, known_walls, explored_areas):
    """A* pathfinding that uses only explored areas and known walls"""
    if goal_grid in known_walls:
        return []
    
    neighbors = [(0,1), (1,0), (0,-1), (-1,0), (1,1), (1,-1), (-1,1), (-1,-1)]
    closed_set = set()
    came_from = {}
    g_score = {start_grid: 0}
    f_score = {start_grid: heuristic(start_grid, goal_grid)}
    open_heap = [(f_score[start_grid], start_grid)]
    
    while open_heap:
        current = heapq.heappop(open_heap)[1]
        
        if current == goal_grid:
            path = []
            while current in came_from:
                path.append(current)
                current = came_from[current]
            return path[::-1]
        
        closed_set.add(current)
        
        for dx, dy in neighbors:
            neighbor = (current[0] + dx, current[1] + dy)
            
            # Check bounds
            if not (0 <= neighbor[0] < WIDTH//GRID_SIZE and 0 <= neighbor[1] < HEIGHT//GRID_SIZE):
                continue
            
            # Only move through explored areas or the goal
            if (neighbor not in explored_areas and neighbor != goal_grid) or neighbor in known_walls or neighbor in closed_set:
                continue
            
            tentative_g = g_score[current] + (1.414 if dx != 0 and dy != 0 else 1)
            
            if tentative_g < g_score.get(neighbor, float('inf')):
                came_from[neighbor] = current
                g_score[neighbor] = tentative_g
                f_score[neighbor] = tentative_g + heuristic(neighbor, goal_grid)
                heapq.heappush(open_heap, (f_score[neighbor], neighbor))
    
    return []

def heuristic(a, b):
    """ฟังก์ชันการประมาณระยะทาง"""
    return math.sqrt((a[0] - b[0])**2 + (a[1] - b[1])**2)

def move_robot():
    """เคลื่อนที่หุ่นยนต์ตามเส้นทาง"""
    global path, robot_pos, robot_angle
    
    if path:
        target_world = get_world_pos(path[0])
        dx = target_world[0] - robot_pos[0]
        dy = target_world[1] - robot_pos[1]
        distance = math.hypot(dx, dy)
        
        if distance < robot_speed:
            path.pop(0)
        else:
            # อัพเดทมุมหุ่นยนต์
            robot_angle = math.degrees(math.atan2(dy, dx))
            
            # เคลื่อนที่
            robot_pos[0] += robot_speed * dx / distance
            robot_pos[1] += robot_speed * dy / distance

def ai_exploration():
    """AI for automatic exploration"""
    global path, exploring, current_target
    
    if not path:  # If no current path
        if len(targets_found) < 3:  # Still need to find more targets
            # Find next safe unexplored area
            next_explore = find_nearest_safe_unexplored()
            if next_explore:
                robot_grid = get_grid_pos(robot_pos)
                path = a_star_pathfind(robot_grid, next_explore, wall_cells, explored_cells)
                if not path:
                    print("No safe path found to unexplored area")
            else:
                print("No more reachable unexplored areas")
        else:
            exploring = False
            print("All 3 targets found! Exploration complete.")

# --- Pygame เริ่มทำงาน ---
pygame.init()
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("AI Robot Explorer - Real-time Mapping")
clock = pygame.time.Clock()
font = pygame.font.SysFont(None, 24)

def draw_text(text, pos, color=BLACK):
    screen.blit(font.render(text, True, color), pos)

# เริ่มต้นด้วยการแสกนจุดเริ่มต้น
scan_area(robot_pos, robot_angle)

while True:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_SPACE:
                exploring = not exploring
                if exploring:
                    print("Starting exploration!")
                else:
                    print("Exploration stopped")
            elif event.key == pygame.K_LEFT and not exploring:
                robot_angle = (robot_angle + 15) % 360
            elif event.key == pygame.K_RIGHT and not exploring:
                robot_angle = (robot_angle - 15) % 360
    
    # วาดแผนที่
    screen.fill(UNEXPLORED_COLOR)  # พื้นหลังเป็นสีดำ (ยังไม่แสกน)
    
    # วาดพื้นที่ที่แสกนแล้ว
    for cell in explored_cells:
        if cell not in wall_cells:
            rect = pygame.Rect(cell[0] * GRID_SIZE, cell[1] * GRID_SIZE, GRID_SIZE, GRID_SIZE)
            pygame.draw.rect(screen, EXPLORED_COLOR, rect)
    
    # วาดกำแพงที่รู้จัก
    for cell in wall_cells:
        rect = pygame.Rect(cell[0] * GRID_SIZE, cell[1] * GRID_SIZE, GRID_SIZE, GRID_SIZE)
        pygame.draw.rect(screen, WALL_COLOR, rect)
    
    # วาดเป้าหมายที่พบแล้ว
    for i in targets_found:
        pos = TARGET_POSITIONS[i]
        pygame.draw.circle(screen, TARGET_COLOR, pos, 12)
        draw_text(str(i+1), (pos[0] - 6, pos[1] - 30), TARGET_COLOR)
    
    # AI สำรวจอัตโนมัติ
    if exploring:
        ai_exploration()
    
    # แสกนพื้นที่รอบๆ
    scan_area(robot_pos, robot_angle)
    check_for_targets()
    
    # เคลื่อนที่
    move_robot()
    
    # วาดเส้นทาง
    for cell in path:
        rect = pygame.Rect(cell[0] * GRID_SIZE, cell[1] * GRID_SIZE, GRID_SIZE, GRID_SIZE)
        pygame.draw.rect(screen, PATH_COLOR, rect)
    
    # วาด FOV
    if robot_pos[0] >= 0 and robot_pos[1] >= 0:
        fov_start_angle = math.radians(robot_angle - FOV_ANGLE/2)
        fov_end_angle = math.radians(robot_angle + FOV_ANGLE/2)
        
        # วาดเส้น FOV
        end_x1 = robot_pos[0] + FOV_RADIUS * math.cos(fov_start_angle)
        end_y1 = robot_pos[1] + FOV_RADIUS * math.sin(fov_start_angle)
        end_x2 = robot_pos[0] + FOV_RADIUS * math.cos(fov_end_angle)
        end_y2 = robot_pos[1] + FOV_RADIUS * math.sin(fov_end_angle)
        
        pygame.draw.line(screen, (0, 255, 0), robot_pos, (end_x1, end_y1), 2)
        pygame.draw.line(screen, (0, 255, 0), robot_pos, (end_x2, end_y2), 2)
    
    # วาดหุ่นยนต์
    pygame.draw.circle(screen, ROBOT_COLOR, (int(robot_pos[0]), int(robot_pos[1])), ROBOT_RADIUS)
    
    # Display information
    draw_text("SPACE = Start/Stop AI Exploration | ← → = Rotate (when stopped)", (10, 10))
    draw_text(f"Targets Found: {len(targets_found)}/3", (10, 35))
    draw_text(f"Explored Area: {len(explored_cells)} cells", (10, 60))
    
    status = "Exploring..." if exploring else "Stopped"
    draw_text(f"Status: {status}", (10, 85))
    
    pygame.display.flip()
    clock.tick(60)

pygame 2.6.1 (SDL 2.28.4, Python 3.10.16)
Hello from the pygame community. https://www.pygame.org/contribute.html
Starting exploration!
Target 1 found!
Target 2 found!
Target 3 found!
All 3 targets found! Exploration complete.


SystemExit: 

c:\Users\earth\anaconda3\envs\Car\lib\site-packages\IPython\core\interactiveshell.py:3587: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [4]:
import pygame
import sys
import heapq
import math
import random
from collections import deque

# --- พื้นฐาน ---
WIDTH, HEIGHT = 800, 600
WHITE, BLACK = (255, 255, 255), (0, 0, 0)
ROBOT_COLOR = (0, 128, 255)
TARGET_COLOR = (255, 0, 0)
WALL_COLOR = (100, 100, 100)
UNEXPLORED_COLOR = (50, 50, 50)
EXPLORED_COLOR = (200, 200, 200)
PATH_COLOR = (0, 200, 255)
WANDER_COLOR = (255, 255, 0)  # สีสำหรับการเดินวน
GRID_SIZE = 20
ROBOT_RADIUS = 10

# --- กำแพง + เป้าหมาย ---
walls = [
    pygame.Rect(200, 150, 200, 20),
    pygame.Rect(500, 150, 200, 20),
    pygame.Rect(200, 450, 200, 20),
    pygame.Rect(600, 450, 150, 20),
    pygame.Rect(200, 150, 20, 300),
    pygame.Rect(580, 50, 20, 500),
    pygame.Rect(50, 300, 150, 20),
]

TARGET_POSITIONS = [
    (150, 500),
    (200, 100),
    (650, 300)
]

# --- ตัวแปรสถานะ ---
robot_pos = [100, 50]
robot_speed = 3
robot_angle = 0
path = []
current_target = None
targets_found = []
exploring = False

# --- ระบบการติดและการเดินวน (ปรับปรุงแล้ว) ---
stuck_counter = 0
last_positions = deque(maxlen=20)  # เพิ่มเป็น 20 ตำแหน่ง
is_wandering = False
wander_direction = 0
wander_timer = 0
wander_duration = 0  # นับเวลาที่เดินวน
max_wander_time = 300  # เดินวนสูงสุด 5 วินาที (300 frames)
exploration_attempts = 0  # นับครั้งที่พยายามหาเป้าหมาย

# --- การแสกนพื้นที่ ---
FOV_RADIUS = 80
FOV_ANGLE = 150
explored_cells = set()
wall_cells = set()

def get_grid_pos(world_pos):
    """แปลงพิกัดโลกเป็นพิกัดกริด"""
    return (int(world_pos[0] // GRID_SIZE), int(world_pos[1] // GRID_SIZE))

def get_world_pos(grid_pos):
    """แปลงพิกัดกริดเป็นพิกัดโลก"""
    return (grid_pos[0] * GRID_SIZE + GRID_SIZE // 2, grid_pos[1] * GRID_SIZE + GRID_SIZE // 2)

def is_wall_at(grid_pos):
    """ตรวจสอบว่าตำแหน่งกริดนั้นเป็นกำแพงหรือไม่"""
    if not (0 <= grid_pos[0] < WIDTH//GRID_SIZE and 0 <= grid_pos[1] < HEIGHT//GRID_SIZE):
        return True  # ขอบจอถือว่าเป็นกำแพง
    
    world_pos = get_world_pos(grid_pos)
    test_rect = pygame.Rect(world_pos[0] - GRID_SIZE//2, world_pos[1] - GRID_SIZE//2, GRID_SIZE, GRID_SIZE)
    return any(test_rect.colliderect(wall) for wall in walls)

def scan_area(robot_pos, robot_angle):
    """แสกนพื้นที่รอบๆ หุ่นยนต์"""
    global explored_cells, wall_cells
    
    robot_grid = get_grid_pos(robot_pos)
    
    # แสกนรอบตัวหุ่นยนต์
    for dx in range(-2, 3):
        for dy in range(-2, 3):
            scan_grid = (robot_grid[0] + dx, robot_grid[1] + dy)
            if 0 <= scan_grid[0] < WIDTH//GRID_SIZE and 0 <= scan_grid[1] < HEIGHT//GRID_SIZE:
                explored_cells.add(scan_grid)
                if is_wall_at(scan_grid):
                    wall_cells.add(scan_grid)
    
    # แสกนใน FOV
    for angle_offset in range(-FOV_ANGLE//2, FOV_ANGLE//2 + 1, 3):
        current_angle = math.radians(robot_angle + angle_offset)
        
        for distance in range(0, FOV_RADIUS, GRID_SIZE//4):
            scan_x = robot_pos[0] + distance * math.cos(current_angle)
            scan_y = robot_pos[1] + distance * math.sin(current_angle)
            
            if not (0 <= scan_x < WIDTH and 0 <= scan_y < HEIGHT):
                break
                
            scan_grid = get_grid_pos((scan_x, scan_y))
            explored_cells.add(scan_grid)
            
            if is_wall_at(scan_grid):
                wall_cells.add(scan_grid)
                break

def check_collision_ahead(pos, angle, distance):
    """ตรวจสอบการชนข้างหน้า"""
    check_x = pos[0] + distance * math.cos(math.radians(angle))
    check_y = pos[1] + distance * math.sin(math.radians(angle))
    
    if not (ROBOT_RADIUS <= check_x <= WIDTH - ROBOT_RADIUS and 
            ROBOT_RADIUS <= check_y <= HEIGHT - ROBOT_RADIUS):
        return True
    
    check_rect = pygame.Rect(check_x - ROBOT_RADIUS, check_y - ROBOT_RADIUS, 
                           ROBOT_RADIUS * 2, ROBOT_RADIUS * 2)
    
    return any(check_rect.colliderect(wall) for wall in walls)

def is_stuck():
    """ตรวจสอบว่าหุ่นยนต์ติดหรือไม่ - ปรับปรุงให้ดีขึ้น"""
    global stuck_counter
    
    if len(last_positions) < 10:
        return False
    
    # ตรวจสอบว่าอยู่ในพื้นที่เดิมมากเกินไป
    recent_positions = list(last_positions)[-10:]
    
    # คำนวณพื้นที่ที่ครอบคลุม
    min_x = min(pos[0] for pos in recent_positions)
    max_x = max(pos[0] for pos in recent_positions)
    min_y = min(pos[1] for pos in recent_positions)
    max_y = max(pos[1] for pos in recent_positions)
    
    coverage_area = (max_x - min_x) * (max_y - min_y)
    
    # ถ้าพื้นที่ครอบคลุมน้อยมาก แสดงว่าติด
    if coverage_area < 1600:  # พื้นที่น้อยกว่า 40x40 พิกเซล
        stuck_counter += 1
    else:
        stuck_counter = max(0, stuck_counter - 3)
    
    return stuck_counter > 45  # ลดเวลาการตรวจสอบ

def find_nearest_unexplored():
    """หาพื้นที่ที่ยังไม่ได้สำรวจที่ใกล้ที่สุด - ปรับปรุงให้ดีขึ้น"""
    robot_grid = get_grid_pos(robot_pos)
    candidates = []
    
    # ขยายขอบเขตการค้นหา
    max_distance = 30  # เพิ่มระยะการค้นหา
    
    # หาขอบเขตการสำรวจ
    for explored in explored_cells:
        for dx, dy in [(0,2), (2,0), (0,-2), (-2,0), (1,1), (-1,1), (1,-1), (-1,-1)]:  # เพิ่มทิศทางแนวทแยง
            neighbor = (explored[0] + dx, explored[1] + dy)
            
            if (0 <= neighbor[0] < WIDTH//GRID_SIZE and 
                0 <= neighbor[1] < HEIGHT//GRID_SIZE and
                neighbor not in explored_cells and
                neighbor not in wall_cells):
                
                dist = math.hypot(neighbor[0] - robot_grid[0], neighbor[1] - robot_grid[1])
                if dist <= max_distance:
                    candidates.append((dist, neighbor))
    
    # หาพื้นที่ที่ไม่ได้สำรวจในบริเวณกว้าง
    if not candidates:
        for x in range(0, WIDTH//GRID_SIZE, 3):
            for y in range(0, HEIGHT//GRID_SIZE, 3):
                grid_pos = (x, y)
                if (grid_pos not in explored_cells and 
                    grid_pos not in wall_cells and
                    not is_wall_at(grid_pos)):
                    
                    dist = math.hypot(x - robot_grid[0], y - robot_grid[1])
                    candidates.append((dist, grid_pos))
    
    if candidates:
        candidates.sort()
        # เลือกจากตัวเลือกที่ดีที่สุด 3 อันดับแรก
        top_candidates = candidates[:min(3, len(candidates))]
        return random.choice(top_candidates)[1]
    
    return None

def smart_wander_behavior():
    """พฤติกรรมการเดินสำรวจแบบฉลาด"""
    global robot_angle, wander_direction, wander_timer, wander_duration
    
    wander_timer += 1
    wander_duration += 1
    
    # เปลี่ยนทิศทางทุก 20-40 เฟรม (เร็วขึ้น)
    if wander_timer > random.randint(20, 40):
        # เลือกทิศทางที่ไปยังพื้นที่ที่ยังไม่ได้สำรวจ
        best_angle = find_best_exploration_angle()
        if best_angle is not None:
            wander_direction = best_angle - robot_angle
        else:
            wander_direction = random.randint(-90, 90)
        wander_timer = 0
    
    # ปรับทิศทาง
    target_angle = (robot_angle + wander_direction) % 360
    angle_diff = (target_angle - robot_angle + 180) % 360 - 180
    
    if abs(angle_diff) > 8:
        robot_angle += 8 if angle_diff > 0 else -8
        robot_angle = robot_angle % 360
    
    # ตรวจสอบการชนก่อนเดิน
    if check_collision_ahead(robot_pos, robot_angle, robot_speed + 15):
        # หาทิศทางที่ดีที่สุดเพื่อหลีกเลี่ยง
        for angle_offset in [45, -45, 90, -90, 135, -135]:
            test_angle = (robot_angle + angle_offset) % 360
            if not check_collision_ahead(robot_pos, test_angle, robot_speed + 15):
                robot_angle = test_angle
                break
        else:
            robot_angle = (robot_angle + 180) % 360  # หมุนกลับ
    else:
        # เดินไปข้างหน้า
        robot_pos[0] += robot_speed * math.cos(math.radians(robot_angle))
        robot_pos[1] += robot_speed * math.sin(math.radians(robot_angle))

def find_best_exploration_angle():
    """หาทิศทางที่ดีที่สุดสำหรับการสำรวจ"""
    robot_grid = get_grid_pos(robot_pos)
    best_angle = None
    max_unexplored = 0
    
    # ตรวจสอบทิศทางต่างๆ
    for angle in range(0, 360, 30):
        unexplored_count = 0
        
        # นับพื้นที่ที่ยังไม่ได้สำรวจในทิศทางนั้น
        for distance in range(3, 10):
            check_x = robot_grid[0] + distance * math.cos(math.radians(angle))
            check_y = robot_grid[1] + distance * math.sin(math.radians(angle))
            check_grid = (int(check_x), int(check_y))
            
            if (0 <= check_grid[0] < WIDTH//GRID_SIZE and 
                0 <= check_grid[1] < HEIGHT//GRID_SIZE):
                if check_grid not in explored_cells and check_grid not in wall_cells:
                    unexplored_count += 1
        
        if unexplored_count > max_unexplored:
            max_unexplored = unexplored_count
            best_angle = angle
    
    return best_angle

def check_for_targets():
    """ตรวจสอบเป้าหมายในพื้นที่ที่แสกน"""
    global targets_found
    
    for i, target_pos in enumerate(TARGET_POSITIONS):
        if i not in targets_found:
            target_grid = get_grid_pos(target_pos)
            if target_grid in explored_cells:
                targets_found.append(i)
                print(f"🎯 Target {i+1} found at {target_pos}!")

def a_star_pathfind(start_grid, goal_grid):
    """A* pathfinding ที่ปรับปรุงแล้ว"""
    if goal_grid in wall_cells:
        return []
    
    neighbors = [(0,1), (1,0), (0,-1), (-1,0), (1,1), (-1,1), (1,-1), (-1,-1)]  # เพิ่มทิศทางแนวทแยง
    closed_set = set()
    came_from = {}
    g_score = {start_grid: 0}
    f_score = {start_grid: heuristic(start_grid, goal_grid)}
    open_heap = [(f_score[start_grid], start_grid)]
    
    while open_heap:
        current = heapq.heappop(open_heap)[1]
        
        if current == goal_grid:
            path = []
            while current in came_from:
                path.append(current)
                current = came_from[current]
            return path[::-1]
        
        closed_set.add(current)
        
        for dx, dy in neighbors:
            neighbor = (current[0] + dx, current[1] + dy)
            
            if not (0 <= neighbor[0] < WIDTH//GRID_SIZE and 0 <= neighbor[1] < HEIGHT//GRID_SIZE):
                continue
            
            if neighbor in wall_cells or neighbor in closed_set:
                continue
            
            # คำนวณต้นทุนการเคลื่อนที่ (แนวทแยงมีต้นทุนสูงกว่า)
            move_cost = 1.4 if abs(dx) + abs(dy) == 2 else 1.0
            tentative_g = g_score[current] + move_cost
            
            if tentative_g < g_score.get(neighbor, float('inf')):
                came_from[neighbor] = current
                g_score[neighbor] = tentative_g
                f_score[neighbor] = tentative_g + heuristic(neighbor, goal_grid)
                heapq.heappush(open_heap, (f_score[neighbor], neighbor))
    
    return []

def heuristic(a, b):
    """ฟังก์ชันการประมาณระยะทาง - ใช้ Euclidean distance"""
    return math.sqrt((a[0] - b[0])**2 + (a[1] - b[1])**2)

def move_robot():
    """เคลื่อนที่หุ่นยนต์ตามเส้นทาง"""
    global path, robot_pos, robot_angle
    
    if path:
        target_world = get_world_pos(path[0])
        dx = target_world[0] - robot_pos[0]
        dy = target_world[1] - robot_pos[1]
        distance = math.hypot(dx, dy)
        
        if distance < robot_speed * 2:
            path.pop(0)
        else:
            robot_angle = math.degrees(math.atan2(dy, dx))
            
            move_x = robot_speed * dx / distance
            move_y = robot_speed * dy / distance
            
            next_x = robot_pos[0] + move_x
            next_y = robot_pos[1] + move_y
            
            # ตรวจสอบการชน
            if not check_collision_ahead(robot_pos, robot_angle, robot_speed + 5):
                robot_pos[0] = next_x
                robot_pos[1] = next_y
            else:
                path.clear()  # ล้างเส้นทางถ้าชน

def ai_exploration():
    """AI สำหรับการสำรวจอัตโนมัติ - ปรับปรุงแล้ว"""
    global path, exploring, is_wandering, stuck_counter, wander_duration, exploration_attempts
    
    # บันทึกตำแหน่งปัจจุบัน
    last_positions.append((robot_pos[0], robot_pos[1]))
    
    # ถ้าอยู่ในโหมดเดินวน
    if is_wandering:
        smart_wander_behavior()
        
        # เงื่อนไขการออกจากโหมดเดินวน
        should_exit_wander = False
        
        # 1. เดินวนเกินเวลากำหนด
        if wander_duration > max_wander_time:
            should_exit_wander = True
            print("⏰ Wander timeout - forcing exploration")
        
        # 2. หาเป้าหมายใหม่ได้และไม่ติดแล้ว
        elif wander_duration > 60:  # เดินวนอย่างน้อย 1 วินาที
            next_target = find_nearest_unexplored()
            if next_target and stuck_counter < 10:
                robot_grid = get_grid_pos(robot_pos)
                test_path = a_star_pathfind(robot_grid, next_target)
                if test_path and len(test_path) > 3:  # เส้นทางต้องยาวพอ
                    should_exit_wander = True
                    path = test_path
                    print(f"🎯 Found viable path to {next_target}! Exiting wander mode...")
        
        if should_exit_wander:
            is_wandering = False
            wander_duration = 0
            stuck_counter = 0
            exploration_attempts = 0
        
        return
    
    # ตรวจสอบการติด
    if is_stuck():
        print("🔄 Robot is stuck! Switching to smart wandering mode...")
        path.clear()
        is_wandering = True
        wander_duration = 0
        stuck_counter = 0
        return
    
    # โหมดปกติ - หาเส้นทางไปยังเป้าหมาย
    if not path and len(targets_found) < 3:
        next_explore = find_nearest_unexplored()
        exploration_attempts += 1
        
        if next_explore:
            robot_grid = get_grid_pos(robot_pos)
            potential_path = a_star_pathfind(robot_grid, next_explore)
            if potential_path:
                path = potential_path
                exploration_attempts = 0
                print(f"🚀 Moving to explore: {next_explore}")
            else:
                # ถ้าหาเส้นทางไม่ได้หลายครั้ง ให้เข้าโหมดเดินวน
                if exploration_attempts > 5:
                    is_wandering = True
                    wander_duration = 0
                    exploration_attempts = 0
                    print("🔄 Multiple pathfinding failures, switching to wander mode...")
        else:
            # ถ้าไม่มีพื้นที่ใหม่ให้สำรวจ ให้เดินวนหาพื้นที่ใหม่
            is_wandering = True
            wander_duration = 0
            print("🔄 No nearby unexplored areas, wandering to find more...")
    elif len(targets_found) >= 3:
        exploring = False
        print("✅ All targets found! Exploration complete.")

# --- Pygame เริ่มทำงาน ---
pygame.init()
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("AI Robot Explorer - Smart Navigation")
clock = pygame.time.Clock()
font = pygame.font.SysFont(None, 24)

def draw_text(text, pos, color=BLACK):
    screen.blit(font.render(text, True, color), pos)

# เริ่มต้น
scan_area(robot_pos, robot_angle)

while True:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_SPACE:
                exploring = not exploring
                if exploring:
                    print("🚀 Starting smart exploration!")
                    is_wandering = False
                    stuck_counter = 0
                    wander_duration = 0
                    exploration_attempts = 0
                    last_positions.clear()
                else:
                    print("⏸️ Exploration stopped")
                    path.clear()
                    is_wandering = False
            elif event.key == pygame.K_LEFT and not exploring:
                robot_angle = (robot_angle + 15) % 360
            elif event.key == pygame.K_RIGHT and not exploring:
                robot_angle = (robot_angle - 15) % 360
            elif event.key == pygame.K_r:
                explored_cells.clear()
                wall_cells.clear()
                targets_found.clear()
                path.clear()
                stuck_counter = 0
                is_wandering = False
                wander_duration = 0
                exploration_attempts = 0
                last_positions.clear()
                print("🔄 Reset exploration!")
    
    # วาดแผนที่
    screen.fill(UNEXPLORED_COLOR)
    
    # วาดพื้นที่ที่สำรวจแล้ว
    for cell in explored_cells:
        if cell not in wall_cells:
            rect = pygame.Rect(cell[0] * GRID_SIZE, cell[1] * GRID_SIZE, GRID_SIZE, GRID_SIZE)
            pygame.draw.rect(screen, EXPLORED_COLOR, rect)
    
    # วาดกำแพง
    for cell in wall_cells:
        rect = pygame.Rect(cell[0] * GRID_SIZE, cell[1] * GRID_SIZE, GRID_SIZE, GRID_SIZE)
        pygame.draw.rect(screen, WALL_COLOR, rect)
    
    # วาดเป้าหมาย
    for i in targets_found:
        pos = TARGET_POSITIONS[i]
        pygame.draw.circle(screen, TARGET_COLOR, pos, 12)
        draw_text(str(i+1), (pos[0] - 6, pos[1] - 30), TARGET_COLOR)
    
    # AI สำรวจ
    if exploring:
        ai_exploration()
    
    # แสกนและตรวจสอบเป้าหมาย
    scan_area(robot_pos, robot_angle)
    check_for_targets()
    
    # เคลื่อนที่
    if not is_wandering:
        move_robot()
    
    # วาดเส้นทาง
    for i, cell in enumerate(path):
        rect = pygame.Rect(cell[0] * GRID_SIZE, cell[1] * GRID_SIZE, GRID_SIZE, GRID_SIZE)
        color = PATH_COLOR if not is_wandering else WANDER_COLOR
        pygame.draw.rect(screen, color, rect)
    
    # วาดตำแหน่งย้อนหลัง (เส้นทางที่เดินมา)
    if len(last_positions) > 1:
        points = list(last_positions)
        for i in range(len(points) - 1):
            alpha = int(255 * (i + 1) / len(points))
            color = (255, 200, 0) if is_wandering else (0, 255, 255)
            if i < len(points) - 1:
                pygame.draw.line(screen, color, points[i], points[i + 1], 2)
    
    # วาด FOV
    fov_start_angle = math.radians(robot_angle - FOV_ANGLE/2)
    fov_end_angle = math.radians(robot_angle + FOV_ANGLE/2)
    
    end_x1 = robot_pos[0] + FOV_RADIUS * math.cos(fov_start_angle)
    end_y1 = robot_pos[1] + FOV_RADIUS * math.sin(fov_start_angle)
    end_x2 = robot_pos[0] + FOV_RADIUS * math.cos(fov_end_angle)
    end_y2 = robot_pos[1] + FOV_RADIUS * math.sin(fov_end_angle)
    
    pygame.draw.line(screen, (0, 255, 0), robot_pos, (end_x1, end_y1), 1)
    pygame.draw.line(screen, (0, 255, 0), robot_pos, (end_x2, end_y2), 1)
    
    # วาดหุ่นยนต์
    color = WANDER_COLOR if is_wandering else ROBOT_COLOR
    pygame.draw.circle(screen, color, (int(robot_pos[0]), int(robot_pos[1])), ROBOT_RADIUS)
    
    # วาดทิศทาง
    front_x = robot_pos[0] + ROBOT_RADIUS * math.cos(math.radians(robot_angle))
    front_y = robot_pos[1] + ROBOT_RADIUS * math.sin(math.radians(robot_angle))
    pygame.draw.line(screen, WHITE, robot_pos, (front_x, front_y), 3)
    
    # แสดงข้อมูล
    draw_text("SPACE = Start/Stop | ← → = Rotate | R = Reset", (10, 10))
    draw_text(f"Targets: {len(targets_found)}/3 🎯", (10, 35))
    draw_text(f"Explored: {len(explored_cells)} cells 🗺️", (10, 60))
    
    status = "Exploring..." if exploring and not is_wandering else "Smart Wandering..." if is_wandering else "Stopped"
    draw_text(f"Status: {status}", (10, 85))
    
    if is_wandering:
        wander_progress = min(100, (wander_duration / max_wander_time) * 100)
        draw_text(f"🔄 Smart Wander Mode ({wander_progress:.0f}%)", (10, 110))
        draw_text(f"Will auto-exit in {max(0, max_wander_time - wander_duration)//60} sec", (10, 135))
    
    pygame.display.flip()
    clock.tick(60)

🚀 Starting smart exploration!
🚀 Moving to explore: (2, 2)
🚀 Moving to explore: (0, 6)
🚀 Moving to explore: (0, 10)
🔄 Robot is stuck! Switching to smart wandering mode...
🎯 Found viable path to (9, 6)! Exiting wander mode...
🎯 Target 2 found at (200, 100)!
🚀 Moving to explore: (11, 9)
🔄 Robot is stuck! Switching to smart wandering mode...
🎯 Found viable path to (18, 9)! Exiting wander mode...
🚀 Moving to explore: (19, 9)
🚀 Moving to explore: (19, 9)
🚀 Moving to explore: (19, 9)
🚀 Moving to explore: (19, 9)
🚀 Moving to explore: (19, 9)
🚀 Moving to explore: (19, 9)
🚀 Moving to explore: (19, 9)
🚀 Moving to explore: (19, 9)
🚀 Moving to explore: (19, 9)
🚀 Moving to explore: (19, 9)
🚀 Moving to explore: (19, 9)
🚀 Moving to explore: (19, 9)
🚀 Moving to explore: (19, 9)
🚀 Moving to explore: (19, 9)
🚀 Moving to explore: (19, 9)
🚀 Moving to explore: (19, 9)
🚀 Moving to explore: (19, 9)
🚀 Moving to explore: (19, 9)
🚀 Moving to explore: (19, 9)
🚀 Moving to explore: (19, 9)
🚀 Moving to explore: (19,

SystemExit: 

In [3]:
import pygame
import sys
import heapq
import math
import random
from collections import deque

# --- พื้นฐาน ---
WIDTH, HEIGHT = 800, 600
WHITE, BLACK = (255, 255, 255), (0, 0, 0)
ROBOT_COLOR = (0, 128, 255)
WALL_COLOR = (100, 100, 100)
UNEXPLORED_COLOR = (50, 50, 50)
EXPLORED_COLOR = (200, 200, 200)
PATH_COLOR = (0, 200, 255)
WANDER_COLOR = (255, 255, 0)
VISITED_PATH_COLOR = (255, 100, 100)  # สีสำหรับเส้นทางที่เดินแล้ว
GRID_SIZE = 20
ROBOT_RADIUS = 10

# --- กำแพง ---
walls = [
    pygame.Rect(200, 150, 200, 20),
    pygame.Rect(500, 150, 200, 20),
    pygame.Rect(200, 450, 200, 20),
    pygame.Rect(600, 450, 150, 20),
    pygame.Rect(200, 150, 20, 300),
    pygame.Rect(580, 50, 20, 500),
    pygame.Rect(50, 300, 150, 20),
]

# --- ตัวแปรสถานะ ---
robot_pos = [100, 50]
robot_speed = 3
robot_angle = 0
path = []
exploring = False

# --- เส้นทางที่เดินแล้ว ---
traveled_path = []  # เก็บเส้นทางที่เดินแล้วทั้งหมด
path_update_counter = 0

# --- ระบบการติดและการเดินวน ---
stuck_counter = 0
last_positions = deque(maxlen=20)
is_wandering = False
wander_direction = 0
wander_timer = 0
wander_duration = 0
max_wander_time = 300
exploration_attempts = 0

# --- การแสกนพื้นที่ ---
FOV_RADIUS = 80
FOV_ANGLE = 150
explored_cells = set()
wall_cells = set()
total_explorable_cells = 0  # จำนวนเซลล์ที่สามารถสำรวจได้ทั้งหมด

def calculate_total_explorable_cells():
    """คำนวณจำนวนเซลล์ที่สามารถสำรวจได้ทั้งหมด"""
    global total_explorable_cells
    total_explorable_cells = 0
    
    for x in range(WIDTH // GRID_SIZE):
        for y in range(HEIGHT // GRID_SIZE):
            if not is_wall_at((x, y)):
                total_explorable_cells += 1

def get_exploration_percentage():
    """คำนวณเปอร์เซ็นต์การสำรวจ"""
    if total_explorable_cells == 0:
        return 0
    explored_non_wall = len([cell for cell in explored_cells if cell not in wall_cells])
    return (explored_non_wall / total_explorable_cells) * 100

def get_grid_pos(world_pos):
    """แปลงพิกัดโลกเป็นพิกัดกริด"""
    return (int(world_pos[0] // GRID_SIZE), int(world_pos[1] // GRID_SIZE))

def get_world_pos(grid_pos):
    """แปลงพิกัดกริดเป็นพิกัดโลก"""
    return (grid_pos[0] * GRID_SIZE + GRID_SIZE // 2, grid_pos[1] * GRID_SIZE + GRID_SIZE // 2)

def is_wall_at(grid_pos):
    """ตรวจสอบว่าตำแหน่งกริดนั้นเป็นกำแพงหรือไม่"""
    if not (0 <= grid_pos[0] < WIDTH//GRID_SIZE and 0 <= grid_pos[1] < HEIGHT//GRID_SIZE):
        return True
    
    world_pos = get_world_pos(grid_pos)
    test_rect = pygame.Rect(world_pos[0] - GRID_SIZE//2, world_pos[1] - GRID_SIZE//2, GRID_SIZE, GRID_SIZE)
    return any(test_rect.colliderect(wall) for wall in walls)

def scan_area(robot_pos, robot_angle):
    """แสกนพื้นที่รอบๆ หุ่นยนต์"""
    global explored_cells, wall_cells
    
    robot_grid = get_grid_pos(robot_pos)
    
    # แสกนรอบตัวหุ่นยนต์
    for dx in range(-2, 3):
        for dy in range(-2, 3):
            scan_grid = (robot_grid[0] + dx, robot_grid[1] + dy)
            if 0 <= scan_grid[0] < WIDTH//GRID_SIZE and 0 <= scan_grid[1] < HEIGHT//GRID_SIZE:
                explored_cells.add(scan_grid)
                if is_wall_at(scan_grid):
                    wall_cells.add(scan_grid)
    
    # แสกนใน FOV
    for angle_offset in range(-FOV_ANGLE//2, FOV_ANGLE//2 + 1, 3):
        current_angle = math.radians(robot_angle + angle_offset)
        
        for distance in range(0, FOV_RADIUS, GRID_SIZE//4):
            scan_x = robot_pos[0] + distance * math.cos(current_angle)
            scan_y = robot_pos[1] + distance * math.sin(current_angle)
            
            if not (0 <= scan_x < WIDTH and 0 <= scan_y < HEIGHT):
                break
                
            scan_grid = get_grid_pos((scan_x, scan_y))
            explored_cells.add(scan_grid)
            
            if is_wall_at(scan_grid):
                wall_cells.add(scan_grid)
                break

def update_traveled_path():
    """อัปเดตเส้นทางที่เดินแล้ว"""
    global path_update_counter, traveled_path
    
    path_update_counter += 1
    
    # บันทึกตำแหน่งทุก 3 เฟรม เพื่อไม่ให้หนาแน่นเกินไป
    if path_update_counter % 3 == 0:
        current_pos = (int(robot_pos[0]), int(robot_pos[1]))
        
        # เช็คว่าตำแหน่งใหม่ห่างจากตำแหน่งล่าสุดพอสมควร
        if not traveled_path or math.hypot(
            current_pos[0] - traveled_path[-1][0], 
            current_pos[1] - traveled_path[-1][1]
        ) > 8:
            traveled_path.append(current_pos)
            
            # จำกัดความยาวของเส้นทาง เพื่อประสิทธิภาพ
            if len(traveled_path) > 2000:
                traveled_path = traveled_path[-1500:]

def check_collision_ahead(pos, angle, distance):
    """ตรวจสอบการชนข้างหน้า"""
    check_x = pos[0] + distance * math.cos(math.radians(angle))
    check_y = pos[1] + distance * math.sin(math.radians(angle))
    
    if not (ROBOT_RADIUS <= check_x <= WIDTH - ROBOT_RADIUS and 
            ROBOT_RADIUS <= check_y <= HEIGHT - ROBOT_RADIUS):
        return True
    
    check_rect = pygame.Rect(check_x - ROBOT_RADIUS, check_y - ROBOT_RADIUS, 
                           ROBOT_RADIUS * 2, ROBOT_RADIUS * 2)
    
    return any(check_rect.colliderect(wall) for wall in walls)

def is_stuck():
    """ตรวจสอบว่าหุ่นยนต์ติดหรือไม่"""
    global stuck_counter
    
    if len(last_positions) < 10:
        return False
    
    recent_positions = list(last_positions)[-10:]
    
    min_x = min(pos[0] for pos in recent_positions)
    max_x = max(pos[0] for pos in recent_positions)
    min_y = min(pos[1] for pos in recent_positions)
    max_y = max(pos[1] for pos in recent_positions)
    
    coverage_area = (max_x - min_x) * (max_y - min_y)
    
    if coverage_area < 1600:
        stuck_counter += 1
    else:
        stuck_counter = max(0, stuck_counter - 3)
    
    return stuck_counter > 45

def find_nearest_unexplored():
    """หาพื้นที่ที่ยังไม่ได้สำรวจที่ใกล้ที่สุด"""
    robot_grid = get_grid_pos(robot_pos)
    candidates = []
    
    max_distance = 30
    
    # หาขอบเขตการสำรวจ
    for explored in explored_cells:
        for dx, dy in [(0,2), (2,0), (0,-2), (-2,0), (1,1), (-1,1), (1,-1), (-1,-1)]:
            neighbor = (explored[0] + dx, explored[1] + dy)
            
            if (0 <= neighbor[0] < WIDTH//GRID_SIZE and 
                0 <= neighbor[1] < HEIGHT//GRID_SIZE and
                neighbor not in explored_cells and
                neighbor not in wall_cells):
                
                dist = math.hypot(neighbor[0] - robot_grid[0], neighbor[1] - robot_grid[1])
                if dist <= max_distance:
                    candidates.append((dist, neighbor))
    
    # หาพื้นที่ที่ไม่ได้สำรวจในบริเวณกว้าง
    if not candidates:
        for x in range(0, WIDTH//GRID_SIZE, 3):
            for y in range(0, HEIGHT//GRID_SIZE, 3):
                grid_pos = (x, y)
                if (grid_pos not in explored_cells and 
                    grid_pos not in wall_cells and
                    not is_wall_at(grid_pos)):
                    
                    dist = math.hypot(x - robot_grid[0], y - robot_grid[1])
                    candidates.append((dist, grid_pos))
    
    if candidates:
        candidates.sort()
        top_candidates = candidates[:min(3, len(candidates))]
        return random.choice(top_candidates)[1]
    
    return None

def smart_wander_behavior():
    """พฤติกรรมการเดินสำรวจแบบฉลาด"""
    global robot_angle, wander_direction, wander_timer, wander_duration
    
    wander_timer += 1
    wander_duration += 1
    
    if wander_timer > random.randint(20, 40):
        best_angle = find_best_exploration_angle()
        if best_angle is not None:
            wander_direction = best_angle - robot_angle
        else:
            wander_direction = random.randint(-90, 90)
        wander_timer = 0
    
    target_angle = (robot_angle + wander_direction) % 360
    angle_diff = (target_angle - robot_angle + 180) % 360 - 180
    
    if abs(angle_diff) > 8:
        robot_angle += 8 if angle_diff > 0 else -8
        robot_angle = robot_angle % 360
    
    if check_collision_ahead(robot_pos, robot_angle, robot_speed + 15):
        for angle_offset in [45, -45, 90, -90, 135, -135]:
            test_angle = (robot_angle + angle_offset) % 360
            if not check_collision_ahead(robot_pos, test_angle, robot_speed + 15):
                robot_angle = test_angle
                break
        else:
            robot_angle = (robot_angle + 180) % 360
    else:
        robot_pos[0] += robot_speed * math.cos(math.radians(robot_angle))
        robot_pos[1] += robot_speed * math.sin(math.radians(robot_angle))

def find_best_exploration_angle():
    """หาทิศทางที่ดีที่สุดสำหรับการสำรวจ"""
    robot_grid = get_grid_pos(robot_pos)
    best_angle = None
    max_unexplored = 0
    
    for angle in range(0, 360, 30):
        unexplored_count = 0
        
        for distance in range(3, 10):
            check_x = robot_grid[0] + distance * math.cos(math.radians(angle))
            check_y = robot_grid[1] + distance * math.sin(math.radians(angle))
            check_grid = (int(check_x), int(check_y))
            
            if (0 <= check_grid[0] < WIDTH//GRID_SIZE and 
                0 <= check_grid[1] < HEIGHT//GRID_SIZE):
                if check_grid not in explored_cells and check_grid not in wall_cells:
                    unexplored_count += 1
        
        if unexplored_count > max_unexplored:
            max_unexplored = unexplored_count
            best_angle = angle
    
    return best_angle

def a_star_pathfind(start_grid, goal_grid):
    """A* pathfinding"""
    if goal_grid in wall_cells:
        return []
    
    neighbors = [(0,1), (1,0), (0,-1), (-1,0), (1,1), (-1,1), (1,-1), (-1,-1)]
    closed_set = set()
    came_from = {}
    g_score = {start_grid: 0}
    f_score = {start_grid: heuristic(start_grid, goal_grid)}
    open_heap = [(f_score[start_grid], start_grid)]
    
    while open_heap:
        current = heapq.heappop(open_heap)[1]
        
        if current == goal_grid:
            path = []
            while current in came_from:
                path.append(current)
                current = came_from[current]
            return path[::-1]
        
        closed_set.add(current)
        
        for dx, dy in neighbors:
            neighbor = (current[0] + dx, current[1] + dy)
            
            if not (0 <= neighbor[0] < WIDTH//GRID_SIZE and 0 <= neighbor[1] < HEIGHT//GRID_SIZE):
                continue
            
            if neighbor in wall_cells or neighbor in closed_set:
                continue
            
            move_cost = 1.4 if abs(dx) + abs(dy) == 2 else 1.0
            tentative_g = g_score[current] + move_cost
            
            if tentative_g < g_score.get(neighbor, float('inf')):
                came_from[neighbor] = current
                g_score[neighbor] = tentative_g
                f_score[neighbor] = tentative_g + heuristic(neighbor, goal_grid)
                heapq.heappush(open_heap, (f_score[neighbor], neighbor))
    
    return []

def heuristic(a, b):
    """ฟังก์ชันการประมาณระยะทาง"""
    return math.sqrt((a[0] - b[0])**2 + (a[1] - b[1])**2)

def move_robot():
    """เคลื่อนที่หุ่นยนต์ตามเส้นทาง"""
    global path, robot_pos, robot_angle
    
    if path:
        target_world = get_world_pos(path[0])
        dx = target_world[0] - robot_pos[0]
        dy = target_world[1] - robot_pos[1]
        distance = math.hypot(dx, dy)
        
        if distance < robot_speed * 2:
            path.pop(0)
        else:
            robot_angle = math.degrees(math.atan2(dy, dx))
            
            move_x = robot_speed * dx / distance
            move_y = robot_speed * dy / distance
            
            next_x = robot_pos[0] + move_x
            next_y = robot_pos[1] + move_y
            
            if not check_collision_ahead(robot_pos, robot_angle, robot_speed + 5):
                robot_pos[0] = next_x
                robot_pos[1] = next_y
            else:
                path.clear()

def is_exploration_complete():
    """ตรวจสอบว่าการสำรวจเสร็จสิ้นแล้วหรือไม่"""
    exploration_percentage = get_exploration_percentage()
    return exploration_percentage >= 100.0  # ถือว่าเสร็จเมื่อสำรวจได้ 95%

def ai_exploration():
    """AI สำหรับการสำรวจอัตโนมัติ"""
    global path, exploring, is_wandering, stuck_counter, wander_duration, exploration_attempts
    
    last_positions.append((robot_pos[0], robot_pos[1]))
    
    # ตรวจสอบว่าการสำรวจเสร็จสิ้นแล้วหรือไม่
    if is_exploration_complete():
        exploring = False
        is_wandering = False
        print("✅ Area exploration complete!")
        return
    
    if is_wandering:
        smart_wander_behavior()
        
        should_exit_wander = False
        
        if wander_duration > max_wander_time:
            should_exit_wander = True
            print("⏰ Wander timeout - forcing exploration")
        
        elif wander_duration > 60:
            next_target = find_nearest_unexplored()
            if next_target and stuck_counter < 10:
                robot_grid = get_grid_pos(robot_pos)
                test_path = a_star_pathfind(robot_grid, next_target)
                if test_path and len(test_path) > 3:
                    should_exit_wander = True
                    path = test_path
                    print(f"🎯 Found viable path to {next_target}! Exiting wander mode...")
        
        if should_exit_wander:
            is_wandering = False
            wander_duration = 0
            stuck_counter = 0
            exploration_attempts = 0
        
        return
    
    if is_stuck():
        print("🔄 Robot is stuck! Switching to smart wandering mode...")
        path.clear()
        is_wandering = True
        wander_duration = 0
        stuck_counter = 0
        return
    
    if not path:
        next_explore = find_nearest_unexplored()
        exploration_attempts += 1
        
        if next_explore:
            robot_grid = get_grid_pos(robot_pos)
            potential_path = a_star_pathfind(robot_grid, next_explore)
            if potential_path:
                path = potential_path
                exploration_attempts = 0
                print(f"🚀 Moving to explore: {next_explore}")
            else:
                if exploration_attempts > 5:
                    is_wandering = True
                    wander_duration = 0
                    exploration_attempts = 0
                    print("🔄 Multiple pathfinding failures, switching to wander mode...")
        else:
            is_wandering = True
            wander_duration = 0
            print("🔄 No nearby unexplored areas, wandering to find more...")

# --- Pygame เริ่มทำงาน ---
pygame.init()
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("AI Robot Area Scanner - Complete Area Mapping")
clock = pygame.time.Clock()
font = pygame.font.SysFont(None, 24)

def draw_text(text, pos, color=BLACK):
    screen.blit(font.render(text, True, color), pos)

# เริ่มต้น
calculate_total_explorable_cells()
scan_area(robot_pos, robot_angle)

while True:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_SPACE:
                exploring = not exploring
                if exploring:
                    print("🚀 Starting area scanning!")
                    is_wandering = False
                    stuck_counter = 0
                    wander_duration = 0
                    exploration_attempts = 0
                    last_positions.clear()
                else:
                    print("⏸️ Area scanning stopped")
                    path.clear()
                    is_wandering = False
            elif event.key == pygame.K_LEFT and not exploring:
                robot_angle = (robot_angle + 15) % 360
            elif event.key == pygame.K_RIGHT and not exploring:
                robot_angle = (robot_angle - 15) % 360
            elif event.key == pygame.K_r:
                explored_cells.clear()
                wall_cells.clear()
                traveled_path.clear()
                path.clear()
                stuck_counter = 0
                is_wandering = False
                wander_duration = 0
                exploration_attempts = 0
                last_positions.clear()
                robot_pos = [100, 50]
                print("🔄 Reset area scanner!")
            elif event.key == pygame.K_c:
                traveled_path.clear()
                print("🧹 Cleared travel path history!")
    
    # วาดแผนที่
    screen.fill(UNEXPLORED_COLOR)
    
    # วาดพื้นที่ที่สำรวจแล้ว
    for cell in explored_cells:
        if cell not in wall_cells:
            rect = pygame.Rect(cell[0] * GRID_SIZE, cell[1] * GRID_SIZE, GRID_SIZE, GRID_SIZE)
            pygame.draw.rect(screen, EXPLORED_COLOR, rect)
    
    # วาดกำแพง
    for cell in wall_cells:
        rect = pygame.Rect(cell[0] * GRID_SIZE, cell[1] * GRID_SIZE, GRID_SIZE, GRID_SIZE)
        pygame.draw.rect(screen, WALL_COLOR, rect)
    
    # วาดเส้นทางที่เดินแล้ว
    if len(traveled_path) > 1:
        # วาดเส้นทางแบบไล่สี (เก่า -> ใหม่)
        for i in range(len(traveled_path) - 1):
            # คำนวณความเข้มสี ตามอายุของเส้นทาง
            age_factor = i / len(traveled_path)
            
            # สีแดงอ่อนสำหรับเส้นทางเก่า สีแดงเข้มสำหรับเส้นทางใหม่
            red_intensity = int(100 + (155 * age_factor))
            color = (red_intensity, 50, 50)
            
            # ความหนาของเส้น (เส้นทางใหม่หนากว่า)
            line_width = max(1, int(2 + (2 * age_factor)))
            
            pygame.draw.line(screen, color, traveled_path[i], traveled_path[i + 1], line_width)
        
        # วาดจุดพิเศษสำหรับจุดเริ่มต้น
        pygame.draw.circle(screen, (0, 255, 0), traveled_path[0], 8)  # จุดเริ่มต้น - สีเขียว
        
        # วาดจุดปัจจุบัน
        if traveled_path:
            pygame.draw.circle(screen, (255, 255, 0), traveled_path[-1], 6)  # จุดปัจจุบัน - สีเหลือง
    
    # AI สำรวจ
    if exploring:
        ai_exploration()
        update_traveled_path()  # อัปเดตเส้นทางที่เดิน
    
    # แสกนและตรวจสอบ
    scan_area(robot_pos, robot_angle)
    
    # เคลื่อนที่
    if not is_wandering:
        move_robot()
    
    # วาดเส้นทางปัจจุบัน
    for i, cell in enumerate(path):
        rect = pygame.Rect(cell[0] * GRID_SIZE, cell[1] * GRID_SIZE, GRID_SIZE, GRID_SIZE)
        color = PATH_COLOR if not is_wandering else WANDER_COLOR
        pygame.draw.rect(screen, color, rect)
    
    # วาด FOV
    fov_start_angle = math.radians(robot_angle - FOV_ANGLE/2)
    fov_end_angle = math.radians(robot_angle + FOV_ANGLE/2)
    
    end_x1 = robot_pos[0] + FOV_RADIUS * math.cos(fov_start_angle)
    end_y1 = robot_pos[1] + FOV_RADIUS * math.sin(fov_start_angle)
    end_x2 = robot_pos[0] + FOV_RADIUS * math.cos(fov_end_angle)
    end_y2 = robot_pos[1] + FOV_RADIUS * math.sin(fov_end_angle)
    
    pygame.draw.line(screen, (0, 255, 0), robot_pos, (end_x1, end_y1), 1)
    pygame.draw.line(screen, (0, 255, 0), robot_pos, (end_x2, end_y2), 1)
    
    # วาดหุ่นยนต์
    color = WANDER_COLOR if is_wandering else ROBOT_COLOR
    pygame.draw.circle(screen, color, (int(robot_pos[0]), int(robot_pos[1])), ROBOT_RADIUS)
    
    # วาดทิศทาง
    front_x = robot_pos[0] + ROBOT_RADIUS * math.cos(math.radians(robot_angle))
    front_y = robot_pos[1] + ROBOT_RADIUS * math.sin(math.radians(robot_angle))
    pygame.draw.line(screen, WHITE, robot_pos, (front_x, front_y), 3)
    
    # แสดงข้อมูล
    draw_text("SPACE = Start/Stop | ← → = Rotate | R = Reset | C = Clear Path", (10, 10))
    
    exploration_pct = get_exploration_percentage()
    draw_text(f"Area Explored: {exploration_pct:.1f}% 🗺️", (10, 35))
    draw_text(f"Cells Scanned: {len([c for c in explored_cells if c not in wall_cells])}/{total_explorable_cells}", (10, 60))
    
    status = "Scanning..." if exploring and not is_wandering else "Smart Wandering..." if is_wandering else "Stopped"
    if is_exploration_complete() and not exploring:
        status = "✅ AREA SCAN COMPLETE!"
    draw_text(f"Status: {status}", (10, 85))
    
    # แสดงข้อมูลเส้นทาง
    if traveled_path:
        total_distance = 0
        for i in range(len(traveled_path) - 1):
            total_distance += math.hypot(
                traveled_path[i+1][0] - traveled_path[i][0],
                traveled_path[i+1][1] - traveled_path[i][1]
            )
        draw_text(f"Travel Distance: {int(total_distance)} pixels 📏", (10, 110))
        draw_text(f"Path Points: {len(traveled_path)} 📍", (10, 135))
    
    if is_wandering:
        wander_progress = min(100, (wander_duration / max_wander_time) * 100)
        draw_text(f"🔄 Smart Wander Mode ({wander_progress:.0f}%)", (10, 160))
    
    pygame.display.flip()
    clock.tick(60)

🚀 Starting area scanning!
🚀 Moving to explore: (2, 2)
🚀 Moving to explore: (0, 6)
🚀 Moving to explore: (0, 10)
🔄 Robot is stuck! Switching to smart wandering mode...
🎯 Found viable path to (7, 8)! Exiting wander mode...
🔄 Robot is stuck! Switching to smart wandering mode...
🎯 Found viable path to (11, 10)! Exiting wander mode...
🚀 Moving to explore: (9, 17)
🔄 Robot is stuck! Switching to smart wandering mode...
🎯 Found viable path to (6, 17)! Exiting wander mode...
🚀 Moving to explore: (7, 17)
🚀 Moving to explore: (8, 17)
🚀 Moving to explore: (10, 17)
🚀 Moving to explore: (11, 17)
🔄 Robot is stuck! Switching to smart wandering mode...
🎯 Found viable path to (13, 22)! Exiting wander mode...
🚀 Moving to explore: (13, 21)
🔄 Robot is stuck! Switching to smart wandering mode...
🎯 Found viable path to (14, 20)! Exiting wander mode...
🚀 Moving to explore: (12, 16)
🔄 Robot is stuck! Switching to smart wandering mode...
🎯 Found viable path to (19, 14)! Exiting wander mode...
🚀 Moving to explore

SystemExit: 

In [2]:
import pygame
import sys
import heapq
import math
import random
from collections import deque

# --- พื้นฐาน ---
WIDTH, HEIGHT = 800, 600
WHITE, BLACK = (255, 255, 255), (0, 0, 0)
ROBOT_COLOR = (0, 128, 255)
WALL_COLOR = (100, 100, 100)
UNEXPLORED_COLOR = (50, 50, 50)
EXPLORED_COLOR = (200, 200, 200)
PATH_COLOR = (0, 200, 255)
WANDER_COLOR = (255, 255, 0)
VISITED_PATH_COLOR = (255, 100, 100)  # สีสำหรับเส้นทางที่เดินแล้ว
GRID_SIZE = 20
ROBOT_RADIUS = 10

# --- กำแพง ---
walls = [
    pygame.Rect(200, 150, 200, 20),
    pygame.Rect(500, 150, 200, 20),
    pygame.Rect(200, 450, 200, 20),
    pygame.Rect(600, 450, 150, 20),
    pygame.Rect(200, 150, 20, 300),
    pygame.Rect(580, 50, 20, 500),
    pygame.Rect(50, 300, 150, 20),
]

# --- ตัวแปรสถานะ ---
robot_pos = [100, 50]
robot_speed = 3
robot_angle = 0
path = []
exploring = False

# --- เส้นทางที่เดินแล้ว ---
traveled_path = []  # เก็บเส้นทางที่เดินแล้วทั้งหมด
path_update_counter = 0

# --- ระบบการติดและการเดินวน ---
stuck_counter = 0
last_positions = deque(maxlen=20)
is_wandering = False
wander_direction = 0
wander_timer = 0
wander_duration = 0
max_wander_time = 180  # ลดเวลาแวนเดอร์ลง
exploration_attempts = 0

# --- การแสกนพื้นที่ (ปรับปรุงแล้ว) ---
FOV_RADIUS = 50  # ลดจาก 80 เป็น 50
FOV_ANGLE = 45   # ลดจาก 150 เป็น 45
explored_cells = set()
wall_cells = set()
total_explorable_cells = 0  # จำนวนเซลล์ที่สามารถสำรวจได้ทั้งหมด

def calculate_total_explorable_cells():
    """คำนวณจำนวนเซลล์ที่สามารถสำรวจได้ทั้งหมด"""
    global total_explorable_cells
    total_explorable_cells = 0
    
    for x in range(WIDTH // GRID_SIZE):
        for y in range(HEIGHT // GRID_SIZE):
            if not is_wall_at((x, y)):
                total_explorable_cells += 1

def get_exploration_percentage():
    """คำนวณเปอร์เซ็นต์การสำรวจ"""
    if total_explorable_cells == 0:
        return 0
    explored_non_wall = len([cell for cell in explored_cells if cell not in wall_cells])
    return (explored_non_wall / total_explorable_cells) * 100

def get_grid_pos(world_pos):
    """แปลงพิกัดโลกเป็นพิกัดกริด"""
    return (int(world_pos[0] // GRID_SIZE), int(world_pos[1] // GRID_SIZE))

def get_world_pos(grid_pos):
    """แปลงพิกัดกริดเป็นพิกัดโลก"""
    return (grid_pos[0] * GRID_SIZE + GRID_SIZE // 2, grid_pos[1] * GRID_SIZE + GRID_SIZE // 2)

def is_wall_at(grid_pos):
    """ตรวจสอบว่าตำแหน่งกริดนั้นเป็นกำแพงหรือไม่"""
    if not (0 <= grid_pos[0] < WIDTH//GRID_SIZE and 0 <= grid_pos[1] < HEIGHT//GRID_SIZE):
        return True
    
    world_pos = get_world_pos(grid_pos)
    test_rect = pygame.Rect(world_pos[0] - GRID_SIZE//2, world_pos[1] - GRID_SIZE//2, GRID_SIZE, GRID_SIZE)
    return any(test_rect.colliderect(wall) for wall in walls)

def scan_area(robot_pos, robot_angle):
    """แสกนพื้นที่รอบๆ หุ่นยนต์ (ปรับปรุงแล้ว)"""
    global explored_cells, wall_cells
    
    robot_grid = get_grid_pos(robot_pos)
    
    # แสกนรอบตัวหุ่นยนต์ (ลดการแสกนรอบตัว)
    for dx in range(-1, 2):
        for dy in range(-1, 2):
            scan_grid = (robot_grid[0] + dx, robot_grid[1] + dy)
            if 0 <= scan_grid[0] < WIDTH//GRID_SIZE and 0 <= scan_grid[1] < HEIGHT//GRID_SIZE:
                explored_cells.add(scan_grid)
                if is_wall_at(scan_grid):
                    wall_cells.add(scan_grid)
    
    # แสกนใน FOV (มุมและระยะลดลงแล้ว)
    for angle_offset in range(-FOV_ANGLE//2, FOV_ANGLE//2 + 1, 5):  # เพิ่มช่วงจาก 3 เป็น 5
        current_angle = math.radians(robot_angle + angle_offset)
        
        for distance in range(0, FOV_RADIUS, GRID_SIZE//3):  # ลดความหนาแน่นการแสกน
            scan_x = robot_pos[0] + distance * math.cos(current_angle)
            scan_y = robot_pos[1] + distance * math.sin(current_angle)
            
            if not (0 <= scan_x < WIDTH and 0 <= scan_y < HEIGHT):
                break
                
            scan_grid = get_grid_pos((scan_x, scan_y))
            explored_cells.add(scan_grid)
            
            if is_wall_at(scan_grid):
                wall_cells.add(scan_grid)
                break

def update_traveled_path():
    """อัปเดตเส้นทางที่เดินแล้ว"""
    global path_update_counter, traveled_path
    
    path_update_counter += 1
    
    # บันทึกตำแหน่งทุก 3 เฟรม เพื่อไม่ให้หนาแน่นเกินไป
    if path_update_counter % 3 == 0:
        current_pos = (int(robot_pos[0]), int(robot_pos[1]))
        
        # เช็คว่าตำแหน่งใหม่ห่างจากตำแหน่งล่าสุดพอสมควร
        if not traveled_path or math.hypot(
            current_pos[0] - traveled_path[-1][0], 
            current_pos[1] - traveled_path[-1][1]
        ) > 8:
            traveled_path.append(current_pos)
            
            # จำกัดความยาวของเส้นทาง เพื่อประสิทธิภาพ
            if len(traveled_path) > 2000:
                traveled_path = traveled_path[-1500:]

def check_collision_ahead(pos, angle, distance):
    """ตรวจสอบการชนข้างหน้า"""
    check_x = pos[0] + distance * math.cos(math.radians(angle))
    check_y = pos[1] + distance * math.sin(math.radians(angle))
    
    if not (ROBOT_RADIUS <= check_x <= WIDTH - ROBOT_RADIUS and 
            ROBOT_RADIUS <= check_y <= HEIGHT - ROBOT_RADIUS):
        return True
    
    check_rect = pygame.Rect(check_x - ROBOT_RADIUS, check_y - ROBOT_RADIUS, 
                           ROBOT_RADIUS * 2, ROBOT_RADIUS * 2)
    
    return any(check_rect.colliderect(wall) for wall in walls)

def is_stuck():
    """ตรวจสอบว่าหุ่นยนต์ติดหรือไม่ (ปรับปรุงการตรวจสอบ)"""
    global stuck_counter
    
    if len(last_positions) < 15:  # เพิ่มจำนวนตำแหน่งที่ต้องตรวจสอบ
        return False
    
    recent_positions = list(last_positions)[-15:]
    
    min_x = min(pos[0] for pos in recent_positions)
    max_x = max(pos[0] for pos in recent_positions)
    min_y = min(pos[1] for pos in recent_positions)
    max_y = max(pos[1] for pos in recent_positions)
    
    coverage_area = (max_x - min_x) * (max_y - min_y)
    
    # ปรับเกณฑ์การติด
    if coverage_area < 1200:  # ลดจาก 1600
        stuck_counter += 1
    else:
        stuck_counter = max(0, stuck_counter - 2)
    
    return stuck_counter > 30  # ลดจาก 45

def find_nearest_unexplored():
    """หาพื้นที่ที่ยังไม่ได้สำรวจที่ใกล้ที่สุด"""
    robot_grid = get_grid_pos(robot_pos)
    candidates = []
    
    max_distance = 25  # ลดจาก 30
    
    # หาขอบเขตการสำรวจ
    for explored in explored_cells:
        for dx, dy in [(0,2), (2,0), (0,-2), (-2,0), (1,1), (-1,1), (1,-1), (-1,-1)]:
            neighbor = (explored[0] + dx, explored[1] + dy)
            
            if (0 <= neighbor[0] < WIDTH//GRID_SIZE and 
                0 <= neighbor[1] < HEIGHT//GRID_SIZE and
                neighbor not in explored_cells and
                neighbor not in wall_cells):
                
                dist = math.hypot(neighbor[0] - robot_grid[0], neighbor[1] - robot_grid[1])
                if dist <= max_distance:
                    candidates.append((dist, neighbor))
    
    # หาพื้นที่ที่ไม่ได้สำรวจในบริเวณกว้าง
    if not candidates:
        for x in range(0, WIDTH//GRID_SIZE, 3):
            for y in range(0, HEIGHT//GRID_SIZE, 3):
                grid_pos = (x, y)
                if (grid_pos not in explored_cells and 
                    grid_pos not in wall_cells and
                    not is_wall_at(grid_pos)):
                    
                    dist = math.hypot(x - robot_grid[0], y - robot_grid[1])
                    candidates.append((dist, grid_pos))
    
    if candidates:
        candidates.sort()
        top_candidates = candidates[:min(3, len(candidates))]
        return random.choice(top_candidates)[1]
    
    return None

def smart_wander_behavior():
    """พฤติกรรมการเดินสำรวจแบบฉลาด (ใช้เฉพาะตอนติด)"""
    global robot_angle, wander_direction, wander_timer, wander_duration
    
    wander_timer += 1
    wander_duration += 1
    
    if wander_timer > random.randint(15, 30):  # ลดเวลา
        best_angle = find_best_exploration_angle()
        if best_angle is not None:
            wander_direction = best_angle - robot_angle
        else:
            wander_direction = random.randint(-60, 60)  # ลดมุมการหมุน
        wander_timer = 0
    
    target_angle = (robot_angle + wander_direction) % 360
    angle_diff = (target_angle - robot_angle + 180) % 360 - 180
    
    if abs(angle_diff) > 6:  # ลดจาก 8
        robot_angle += 6 if angle_diff > 0 else -6
        robot_angle = robot_angle % 360
    
    if check_collision_ahead(robot_pos, robot_angle, robot_speed + 15):
        for angle_offset in [45, -45, 90, -90]:  # ลดตัวเลือกมุม
            test_angle = (robot_angle + angle_offset) % 360
            if not check_collision_ahead(robot_pos, test_angle, robot_speed + 15):
                robot_angle = test_angle
                break
        else:
            robot_angle = (robot_angle + 180) % 360
    else:
        robot_pos[0] += robot_speed * math.cos(math.radians(robot_angle))
        robot_pos[1] += robot_speed * math.sin(math.radians(robot_angle))

def find_best_exploration_angle():
    """หาทิศทางที่ดีที่สุดสำหรับการสำรวจ (ปรับปรุงแล้ว)"""
    robot_grid = get_grid_pos(robot_pos)
    best_angle = None
    max_unexplored = 0
    
    for angle in range(0, 360, 45):  # เพิ่มช่วงจาก 30 เป็น 45
        unexplored_count = 0
        
        for distance in range(2, 8):  # ลดระยะตรวจสอบ
            check_x = robot_grid[0] + distance * math.cos(math.radians(angle))
            check_y = robot_grid[1] + distance * math.sin(math.radians(angle))
            check_grid = (int(check_x), int(check_y))
            
            if (0 <= check_grid[0] < WIDTH//GRID_SIZE and 
                0 <= check_grid[1] < HEIGHT//GRID_SIZE):
                if check_grid not in explored_cells and check_grid not in wall_cells:
                    unexplored_count += 1
        
        if unexplored_count > max_unexplored:
            max_unexplored = unexplored_count
            best_angle = angle
    
    return best_angle

def a_star_pathfind(start_grid, goal_grid):
    """A* pathfinding"""
    if goal_grid in wall_cells:
        return []
    
    neighbors = [(0,1), (1,0), (0,-1), (-1,0), (1,1), (-1,1), (1,-1), (-1,-1)]
    closed_set = set()
    came_from = {}
    g_score = {start_grid: 0}
    f_score = {start_grid: heuristic(start_grid, goal_grid)}
    open_heap = [(f_score[start_grid], start_grid)]
    
    while open_heap:
        current = heapq.heappop(open_heap)[1]
        
        if current == goal_grid:
            path = []
            while current in came_from:
                path.append(current)
                current = came_from[current]
            return path[::-1]
        
        closed_set.add(current)
        
        for dx, dy in neighbors:
            neighbor = (current[0] + dx, current[1] + dy)
            
            if not (0 <= neighbor[0] < WIDTH//GRID_SIZE and 0 <= neighbor[1] < HEIGHT//GRID_SIZE):
                continue
            
            if neighbor in wall_cells or neighbor in closed_set:
                continue
            
            move_cost = 1.4 if abs(dx) + abs(dy) == 2 else 1.0
            tentative_g = g_score[current] + move_cost
            
            if tentative_g < g_score.get(neighbor, float('inf')):
                came_from[neighbor] = current
                g_score[neighbor] = tentative_g
                f_score[neighbor] = tentative_g + heuristic(neighbor, goal_grid)
                heapq.heappush(open_heap, (f_score[neighbor], neighbor))
    
    return []

def heuristic(a, b):
    """ฟังก์ชันการประมาณระยะทาง"""
    return math.sqrt((a[0] - b[0])**2 + (a[1] - b[1])**2)

def move_robot():
    """เคลื่อนที่หุ่นยนต์ตามเส้นทาง"""
    global path, robot_pos, robot_angle
    
    if path:
        target_world = get_world_pos(path[0])
        dx = target_world[0] - robot_pos[0]
        dy = target_world[1] - robot_pos[1]
        distance = math.hypot(dx, dy)
        
        if distance < robot_speed * 2:
            path.pop(0)
        else:
            robot_angle = math.degrees(math.atan2(dy, dx))
            
            move_x = robot_speed * dx / distance
            move_y = robot_speed * dy / distance
            
            next_x = robot_pos[0] + move_x
            next_y = robot_pos[1] + move_y
            
            if not check_collision_ahead(robot_pos, robot_angle, robot_speed + 5):
                robot_pos[0] = next_x
                robot_pos[1] = next_y
            else:
                path.clear()

def is_exploration_complete():
    """ตรวจสอบว่าการสำรวจเสร็จสิ้นแล้วหรือไม่"""
    exploration_percentage = get_exploration_percentage()
    return exploration_percentage >= 100.0  # 

def ai_exploration():
    """AI สำหรับการสำรวจอัตโนมัติ (ปรับปรุงแล้ว)"""
    global path, exploring, is_wandering, stuck_counter, wander_duration, exploration_attempts
    
    last_positions.append((robot_pos[0], robot_pos[1]))
    
    # ตรวจสอบว่าการสำรวจเสร็จสิ้นแล้วหรือไม่
    if is_exploration_complete():
        exploring = False
        is_wandering = False
        print("✅ Area exploration complete!")
        return
    
    # เดินวนเฉพาะตอนติด
    if is_wandering:
        smart_wander_behavior()
        
        # ออกจากโหมดเดินวนเมื่อ:
        # 1. หมดเวลา
        # 2. หาเป้าหมายใหม่ได้และไม่ติดแล้ว
        if wander_duration > max_wander_time:
            is_wandering = False
            wander_duration = 0
            stuck_counter = 0
            print("⏰ Wander timeout - back to normal exploration")
        elif wander_duration > 30:  # ลดจาก 60
            next_target = find_nearest_unexplored()
            if next_target:
                robot_grid = get_grid_pos(robot_pos)
                test_path = a_star_pathfind(robot_grid, next_target)
                if test_path and len(test_path) > 2:  # ลดจาก 3
                    is_wandering = False
                    wander_duration = 0
                    stuck_counter = 0
                    path = test_path
                    print(f"🎯 Found path to {next_target}! Exiting wander mode...")
        
        return
    
    # เข้าสู่โหมดเดินวนเฉพาะตอนติด
    if is_stuck():
        print("🔄 Robot is stuck! Switching to wander mode...")
        path.clear()
        is_wandering = True
        wander_duration = 0
        stuck_counter = 0
        return
    
    # การสำรวจปกติ
    if not path:
        next_explore = find_nearest_unexplored()
        exploration_attempts += 1
        
        if next_explore:
            robot_grid = get_grid_pos(robot_pos)
            potential_path = a_star_pathfind(robot_grid, next_explore)
            if potential_path:
                path = potential_path
                exploration_attempts = 0
                print(f"🚀 Moving to explore: {next_explore}")
            else:
                if exploration_attempts > 3:  # ลดจาก 5
                    is_wandering = True
                    wander_duration = 0
                    exploration_attempts = 0
                    print("🔄 Pathfinding failed, switching to wander mode...")
        else:
            # ไม่มีพื้นที่ใหม่ให้สำรวจแล้ว
            print("🏁 No more unexplored areas found!")
            exploring = False

# --- Pygame เริ่มทำงาน ---
pygame.init()
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("AI Robot Area Scanner - Optimized (45° FOV, 50px Range)")
clock = pygame.time.Clock()
font = pygame.font.SysFont(None, 24)

def draw_text(text, pos, color=BLACK):
    screen.blit(font.render(text, True, color), pos)

# เริ่มต้น
calculate_total_explorable_cells()
scan_area(robot_pos, robot_angle)

while True:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_SPACE:
                exploring = not exploring
                if exploring:
                    print("🚀 Starting optimized area scanning!")
                    is_wandering = False
                    stuck_counter = 0
                    wander_duration = 0
                    exploration_attempts = 0
                    last_positions.clear()
                else:
                    print("⏸️ Area scanning stopped")
                    path.clear()
                    is_wandering = False
            elif event.key == pygame.K_LEFT and not exploring:
                robot_angle = (robot_angle + 15) % 360
            elif event.key == pygame.K_RIGHT and not exploring:
                robot_angle = (robot_angle - 15) % 360
            elif event.key == pygame.K_r:
                explored_cells.clear()
                wall_cells.clear()
                traveled_path.clear()
                path.clear()
                stuck_counter = 0
                is_wandering = False
                wander_duration = 0
                exploration_attempts = 0
                last_positions.clear()
                robot_pos = [100, 50]
                print("🔄 Reset area scanner!")
            elif event.key == pygame.K_c:
                traveled_path.clear()
                print("🧹 Cleared travel path history!")
    
    # วาดแผนที่
    screen.fill(UNEXPLORED_COLOR)
    
    # วาดพื้นที่ที่สำรวจแล้ว
    for cell in explored_cells:
        if cell not in wall_cells:
            rect = pygame.Rect(cell[0] * GRID_SIZE, cell[1] * GRID_SIZE, GRID_SIZE, GRID_SIZE)
            pygame.draw.rect(screen, EXPLORED_COLOR, rect)
    
    # วาดกำแพง
    for cell in wall_cells:
        rect = pygame.Rect(cell[0] * GRID_SIZE, cell[1] * GRID_SIZE, GRID_SIZE, GRID_SIZE)
        pygame.draw.rect(screen, WALL_COLOR, rect)
    
    # วาดเส้นทางที่เดินแล้ว
    if len(traveled_path) > 1:
        # วาดเส้นทางแบบไล่สี (เก่า -> ใหม่)
        for i in range(len(traveled_path) - 1):
            # คำนวณความเข้มสี ตามอายุของเส้นทาง
            age_factor = i / len(traveled_path)
            
            # สีแดงอ่อนสำหรับเส้นทางเก่า สีแดงเข้มสำหรับเส้นทางใหม่
            red_intensity = int(100 + (155 * age_factor))
            color = (red_intensity, 50, 50)
            
            # ความหนาของเส้น (เส้นทางใหม่หนากว่า)
            line_width = max(1, int(2 + (2 * age_factor)))
            
            pygame.draw.line(screen, color, traveled_path[i], traveled_path[i + 1], line_width)
        
        # วาดจุดพิเศษสำหรับจุดเริ่มต้น
        pygame.draw.circle(screen, (0, 255, 0), traveled_path[0], 8)  # จุดเริ่มต้น - สีเขียว
        
        # วาดจุดปัจจุบัน
        if traveled_path:
            pygame.draw.circle(screen, (255, 255, 0), traveled_path[-1], 6)  # จุดปัจจุบัน - สีเหลือง
    
    # AI สำรวจ
    if exploring:
        ai_exploration()
        update_traveled_path()  # อัปเดตเส้นทางที่เดิน
    
    # แสกนและตรวจสอบ
    scan_area(robot_pos, robot_angle)
    
    # เคลื่อนที่
    if not is_wandering:
        move_robot()
    
    # วาดเส้นทางปัจจุบัน
    for i, cell in enumerate(path):
        rect = pygame.Rect(cell[0] * GRID_SIZE, cell[1] * GRID_SIZE, GRID_SIZE, GRID_SIZE)
        color = PATH_COLOR if not is_wandering else WANDER_COLOR
        pygame.draw.rect(screen, color, rect)
    
    # วาด FOV (ปรับปรุงแล้ว - มุม 45 องศา ระยะ 50)
    fov_start_angle = math.radians(robot_angle - FOV_ANGLE/2)
    fov_end_angle = math.radians(robot_angle + FOV_ANGLE/2)
    
    end_x1 = robot_pos[0] + FOV_RADIUS * math.cos(fov_start_angle)
    end_y1 = robot_pos[1] + FOV_RADIUS * math.sin(fov_start_angle)
    end_x2 = robot_pos[0] + FOV_RADIUS * math.cos(fov_end_angle)
    end_y2 = robot_pos[1] + FOV_RADIUS * math.sin(fov_end_angle)
    
    pygame.draw.line(screen, (0, 255, 0), robot_pos, (end_x1, end_y1), 1)
    pygame.draw.line(screen, (0, 255, 0), robot_pos, (end_x2, end_y2), 1)
    
    # วาดหุ่นยนต์
    color = WANDER_COLOR if is_wandering else ROBOT_COLOR
    pygame.draw.circle(screen, color, (int(robot_pos[0]), int(robot_pos[1])), ROBOT_RADIUS)
    
    # วาดทิศทาง
    front_x = robot_pos[0] + ROBOT_RADIUS * math.cos(math.radians(robot_angle))
    front_y = robot_pos[1] + ROBOT_RADIUS * math.sin(math.radians(robot_angle))
    pygame.draw.line(screen, WHITE, robot_pos, (front_x, front_y), 3)
    
    # แสดงข้อมูล
    draw_text("SPACE = Start/Stop | ← → = Rotate | R = Reset | C = Clear Path", (10, 10))
    
    exploration_pct = get_exploration_percentage()
    draw_text(f"Area Explored: {exploration_pct:.1f}% 🗺️", (10, 35))
    draw_text(f"Cells Scanned: {len([c for c in explored_cells if c not in wall_cells])}/{total_explorable_cells}", (10, 60))
    
    status = "Scanning..." if exploring and not is_wandering else "Smart Wandering..." if is_wandering else "Stopped"
    if is_exploration_complete() and not exploring:
        status = "✅ AREA SCAN COMPLETE!"
    draw_text(f"Status: {status}", (10, 85))
    
    # แสดงข้อมูลเส้นทาง
    if traveled_path:
        total_distance = 0
        for i in range(len(traveled_path) - 1):
            total_distance += math.hypot(
                traveled_path[i+1][0] - traveled_path[i][0],
                traveled_path[i+1][1] - traveled_path[i][1]
            )
        draw_text(f"Travel Distance: {int(total_distance)} pixels 📏", (10, 110))
        draw_text(f"Path Points: {len(traveled_path)} 📍", (10, 135))
    
    if is_wandering:
        wander_progress = min(100, (wander_duration / max_wander_time) * 100)
        draw_text(f"🔄 Smart Wander Mode ({wander_progress:.0f}%)", (10, 160))
    
    pygame.display.flip()
    clock.tick(60)

🚀 Starting optimized area scanning!
🚀 Moving to explore: (3, 2)
🚀 Moving to explore: (3, 0)
🚀 Moving to explore: (1, 0)
🚀 Moving to explore: (0, 1)
🔄 Robot is stuck! Switching to wander mode...
🎯 Found path to (2, 7)! Exiting wander mode...
🚀 Moving to explore: (0, 7)
🔄 Robot is stuck! Switching to wander mode...
🎯 Found path to (0, 13)! Exiting wander mode...
🚀 Moving to explore: (2, 15)
🔄 Robot is stuck! Switching to wander mode...
🎯 Found path to (5, 14)! Exiting wander mode...
🚀 Moving to explore: (5, 14)
🚀 Moving to explore: (6, 14)
🔄 Robot is stuck! Switching to wander mode...
🎯 Found path to (11, 19)! Exiting wander mode...
🚀 Moving to explore: (11, 21)
🔄 Robot is stuck! Switching to wander mode...
🎯 Found path to (12, 22)! Exiting wander mode...
🚀 Moving to explore: (14, 22)
🔄 Robot is stuck! Switching to wander mode...
🎯 Found path to (18, 22)! Exiting wander mode...
🚀 Moving to explore: (18, 22)
🚀 Moving to explore: (18, 22)
🚀 Moving to explore: (18, 22)
🚀 Moving to explore: 

SystemExit: 

In [1]:
import pygame
import math
import sys
from enum import Enum
from collections import deque
import random
import heapq

# Constants
WIDTH, HEIGHT = 1200, 800
GRID_SIZE = 20
ZONE_SIZE = 120
ROBOT_RADIUS = 12
ROBOT_SPEED = 3
FOV_ANGLE = 90
FOV_RADIUS = 80

# Colors
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
WALL_COLOR = (64, 64, 64)
ROBOT_COLOR = (255, 100, 100)
WANDER_COLOR = (255, 165, 0)
EXPLORED_COLOR = (100, 200, 255)
UNEXPLORED_COLOR = (240, 240, 240)
ZONE_BORDER_COLOR = (100, 100, 100)
CURRENT_ZONE_COLOR = (255, 255, 0)
COMPLETED_ZONE_COLOR = (0, 255, 0)
PATH_COLOR = (255, 0, 255)

# Robot States
class RobotState(Enum):
    EXPLORING = "EXPLORING"
    WANDERING = "WANDERING"
    STUCK = "STUCK"

# Global Variables
robot_pos = [100, 50]
robot_angle = 0
explored_cells = set()
wall_cells = set()
traveled_path = []
walls = [
    (200, 100, 400, 20), (300, 200, 20, 200), (500, 150, 200, 20),
    (150, 400, 300, 20), (600, 300, 20, 200), (800, 100, 20, 300),
    (100, 600, 500, 20), (700, 500, 200, 20), (900, 200, 20, 400)
]

# Zone system variables
zones = {}
current_zone = None
total_explorable_cells = 0

# AI variables
exploring = False
is_wandering = False
wander_duration = 0
max_wander_time = 180
stuck_counter = 0
path = deque()
current_path_visual = []  # For visualizing A* path
last_positions = deque(maxlen=20)
exploration_attempts = 0
robot_state = RobotState.EXPLORING

# --- Pygame Initialization ---
pygame.init()
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Enhanced AI Robot Zone Scanner with A* v3.0")
clock = pygame.time.Clock()
font = pygame.font.SysFont(None, 24)
small_font = pygame.font.SysFont(None, 18)
large_font = pygame.font.SysFont(None, 32)

def draw_text(text, pos, color=BLACK, font_obj=font):
    screen.blit(font_obj.render(text, True, color), pos)

def get_cell_pos(world_pos):
    """Convert world position to cell coordinates"""
    return (int(world_pos[0] // GRID_SIZE), int(world_pos[1] // GRID_SIZE))

def get_world_pos(cell):
    """Convert cell coordinates to world position"""
    return (cell[0] * GRID_SIZE + GRID_SIZE // 2, cell[1] * GRID_SIZE + GRID_SIZE // 2)

def is_wall_at_position(pos):
    """Check if there's a wall at the given position"""
    for wall in walls:
        if pygame.Rect(wall).collidepoint(pos):
            return True
    return False

def is_cell_walkable(cell):
    """Check if a cell is walkable (not a wall and within bounds)"""
    world_pos = get_world_pos(cell)
    return (0 <= world_pos[0] < WIDTH and 
            0 <= world_pos[1] < HEIGHT and 
            not is_wall_at_position(world_pos) and
            cell not in wall_cells)

def get_neighbors(cell):
    """Get walkable neighboring cells (8-directional)"""
    x, y = cell
    neighbors = []
    
    # 8-directional movement
    directions = [(-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1), (1, 0), (1, 1)]
    
    for dx, dy in directions:
        new_cell = (x + dx, y + dy)
        if is_cell_walkable(new_cell):
            neighbors.append(new_cell)
    
    return neighbors

def heuristic(cell1, cell2):
    """Euclidean distance heuristic for A*"""
    return math.sqrt((cell1[0] - cell2[0])**2 + (cell1[1] - cell2[1])**2)

def astar_pathfind(start_cell, goal_cell):
    """A* pathfinding algorithm"""
    if not is_cell_walkable(goal_cell):
        return []
    
    open_set = [(0, start_cell)]
    came_from = {}
    g_score = {start_cell: 0}
    f_score = {start_cell: heuristic(start_cell, goal_cell)}
    
    while open_set:
        current = heapq.heappop(open_set)[1]
        
        if current == goal_cell:
            # Reconstruct path
            path = []
            while current in came_from:
                path.append(get_world_pos(current))
                current = came_from[current]
            path.append(get_world_pos(start_cell))
            return path[::-1]  # Reverse to get start->goal path
        
        for neighbor in get_neighbors(current):
            # Calculate movement cost (diagonal movement costs more)
            dx = abs(neighbor[0] - current[0])
            dy = abs(neighbor[1] - current[1])
            move_cost = 1.414 if dx == 1 and dy == 1 else 1.0  # √2 for diagonal
            
            tentative_g_score = g_score[current] + move_cost
            
            if neighbor not in g_score or tentative_g_score < g_score[neighbor]:
                came_from[neighbor] = current
                g_score[neighbor] = tentative_g_score
                f_score[neighbor] = tentative_g_score + heuristic(neighbor, goal_cell)
                
                if neighbor not in [item[1] for item in open_set]:
                    heapq.heappush(open_set, (f_score[neighbor], neighbor))
    
    return []  # No path found

def find_best_unexplored_target():
    """Find the best unexplored target using A* pathfinding strategy"""
    current_zone_key = get_current_zone(robot_pos)
    robot_cell = get_cell_pos(robot_pos)
    
    # Priority 1: Unexplored cells in current zone (if zone is not 100% complete)
    if current_zone_key and current_zone_key in zones and not zones[current_zone_key]["completed"]:
        zone_unexplored = []
        for cell in zones[current_zone_key]["cells"]:
            if cell not in explored_cells and cell not in wall_cells and is_cell_walkable(cell):
                zone_unexplored.append(cell)
        
        if zone_unexplored:
            # Find closest unexplored cell in current zone
            closest_cell = min(zone_unexplored, 
                             key=lambda cell: heuristic(robot_cell, cell))
            
            # Use A* to find path
            path_points = astar_pathfind(robot_cell, closest_cell)
            if path_points:
                print(f"🎯 A* path to unexplored cell in current zone {current_zone_key}")
                return path_points, current_zone_key
    
    # Priority 2: Find nearest incomplete zone
    incomplete_zones = [(zone_key, zone_data) for zone_key, zone_data in zones.items() 
                       if not zone_data["completed"]]
    
    if incomplete_zones:
        best_path = None
        best_zone = None
        min_path_length = float('inf')
        
        for zone_key, zone_data in incomplete_zones:
            # Find unexplored cells in this zone
            zone_unexplored = []
            for cell in zone_data["cells"]:
                if cell not in explored_cells and cell not in wall_cells and is_cell_walkable(cell):
                    zone_unexplored.append(cell)
            
            if zone_unexplored:
                # Find closest unexplored cell in this zone
                closest_cell = min(zone_unexplored,
                                 key=lambda cell: heuristic(robot_cell, cell))
                
                # Use A* to find path
                path_points = astar_pathfind(robot_cell, closest_cell)
                if path_points and len(path_points) < min_path_length:
                    min_path_length = len(path_points)
                    best_path = path_points
                    best_zone = zone_key
        
        if best_path:
            print(f"🚀 A* path to nearest incomplete zone {best_zone} (distance: {min_path_length})")
            return best_path, best_zone
    
    return None, None

def scan_area(pos, angle):
    """Scan the area around the robot and update explored/wall cells"""
    robot_cell = get_cell_pos(pos)
    explored_cells.add(robot_cell)
    
    # Add current position to path - never remove points
    if not traveled_path or (traveled_path[-1][0] != int(pos[0]) or traveled_path[-1][1] != int(pos[1])):
        traveled_path.append((int(pos[0]), int(pos[1])))
    
    # Scan within FOV
    fov_start = math.radians(angle - FOV_ANGLE/2)
    fov_end = math.radians(angle + FOV_ANGLE/2)
    
    for distance in range(0, FOV_RADIUS, GRID_SIZE // 2):
        for angle_offset in range(-FOV_ANGLE//2, FOV_ANGLE//2 + 1, 5):
            scan_angle = math.radians(angle + angle_offset)
            scan_x = pos[0] + distance * math.cos(scan_angle)
            scan_y = pos[1] + distance * math.sin(scan_angle)
            
            if 0 <= scan_x < WIDTH and 0 <= scan_y < HEIGHT:
                scan_cell = get_cell_pos((scan_x, scan_y))
                
                if is_wall_at_position((scan_x, scan_y)):
                    wall_cells.add(scan_cell)
                else:
                    explored_cells.add(scan_cell)

def calculate_zone_cells():
    """Calculate explorable cells in each zone"""
    zones.clear()
    
    zone_width = ZONE_SIZE
    zone_height = ZONE_SIZE
    
    for zone_x in range(0, WIDTH, zone_width):
        for zone_y in range(0, HEIGHT, zone_height):
            zone_key = (zone_x // zone_width, zone_y // zone_height)
            
            explorable_count = 0
            zone_cells = []
            
            for x in range(zone_x, min(zone_x + zone_width, WIDTH), GRID_SIZE):
                for y in range(zone_y, min(zone_y + zone_height, HEIGHT), GRID_SIZE):
                    cell_x = x // GRID_SIZE
                    cell_y = y // GRID_SIZE
                    cell = (cell_x, cell_y)
                    
                    world_pos = (x, y)
                    is_wall = False
                    
                    for wall in walls:
                        if pygame.Rect(wall).collidepoint(world_pos):
                            is_wall = True
                            break
                    
                    if not is_wall:
                        explorable_count += 1
                        zone_cells.append(cell)
            
            if explorable_count > 0:
                zones[zone_key] = {
                    "explorable_cells": explorable_count,
                    "explored_cells": 0,
                    "completed": False,
                    "cells": zone_cells,
                    "bounds": (zone_x, zone_y, zone_x + zone_width, zone_y + zone_height)
                }
    
    print(f"🗺️ Created {len(zones)} zones")

def get_current_zone(position):
    """Get current zone from position"""
    zone_x = int(position[0]) // ZONE_SIZE
    zone_y = int(position[1]) // ZONE_SIZE
    zone_key = (zone_x, zone_y)
    
    if zone_key in zones:
        return zone_key
    return None

def update_zone_exploration():
    """Update exploration status of each zone"""
    global current_zone
    
    new_zone = get_current_zone(robot_pos)
    
    if new_zone != current_zone:
        if current_zone is not None:
            print(f"🚪 Left zone {current_zone}")
        if new_zone is not None:
            print(f"🎯 Entered zone {new_zone}")
        current_zone = new_zone
    
    for zone_key, zone_data in zones.items():
        explored_in_zone = 0
        for cell in zone_data["cells"]:
            if cell in explored_cells and cell not in wall_cells:
                explored_in_zone += 1
        
        zone_data["explored_cells"] = explored_in_zone
        
        if explored_in_zone >= zone_data["explorable_cells"]:
            if not zone_data["completed"]:
                zone_data["completed"] = True
                print(f"✅ Zone {zone_key} completed! ({explored_in_zone}/{zone_data['explorable_cells']} cells - 100%)")

def is_exploration_complete():
    """Check if exploration is complete"""
    if not zones:
        return False
        
    completed_zones = sum(1 for zone_data in zones.values() if zone_data["completed"])
    total_zones = len(zones)
    
    return completed_zones >= total_zones

def calculate_total_explorable_cells():
    """Calculate total explorable cells"""
    global total_explorable_cells
    
    calculate_zone_cells()
    total_explorable_cells = sum(zone_data["explorable_cells"] for zone_data in zones.values())
    print(f"📊 Total explorable cells: {total_explorable_cells}")

def move_robot():
    """Move robot along the current path"""
    global robot_pos, robot_angle, stuck_counter
    
    if not path:
        return
    
    target = path[0]
    dx = target[0] - robot_pos[0]
    dy = target[1] - robot_pos[1]
    distance = math.hypot(dx, dy)
    
    if distance < ROBOT_SPEED:
        robot_pos = list(target)
        path.popleft()
        stuck_counter = 0
    else:
        robot_pos[0] += (dx / distance) * ROBOT_SPEED
        robot_pos[1] += (dy / distance) * ROBOT_SPEED
        
        robot_angle = math.degrees(math.atan2(dy, dx))
        
        if len(last_positions) >= 10:
            avg_distance = sum(math.hypot(pos[0] - robot_pos[0], pos[1] - robot_pos[1]) 
                             for pos in last_positions) / len(last_positions)
            if avg_distance < ROBOT_SPEED:
                stuck_counter += 1
                if stuck_counter > 30:
                    start_wandering()

def start_wandering():
    """Start wandering mode"""
    global is_wandering, wander_duration, robot_state, path, current_path_visual
    is_wandering = True
    wander_duration = 0
    robot_state = RobotState.WANDERING
    path.clear()
    current_path_visual.clear()
    print("🔄 Starting smart wandering mode")

def smart_wander():
    """Smart wandering behavior"""
    global robot_pos, robot_angle, wander_duration, is_wandering
    
    wander_duration += 1
    
    if wander_duration % 20 == 0:
        robot_angle += random.randint(-45, 45)
        robot_angle %= 360
    
    new_x = robot_pos[0] + ROBOT_SPEED * math.cos(math.radians(robot_angle))
    new_y = robot_pos[1] + ROBOT_SPEED * math.sin(math.radians(robot_angle))
    
    if (ROBOT_RADIUS <= new_x <= WIDTH - ROBOT_RADIUS and 
        ROBOT_RADIUS <= new_y <= HEIGHT - ROBOT_RADIUS and
        not is_wall_at_position((new_x, new_y))):
        robot_pos[0] = new_x
        robot_pos[1] = new_y
    else:
        robot_angle = (robot_angle + 90) % 360
    
    if wander_duration > max_wander_time:
        is_wandering = False
        wander_duration = 0
        robot_state = RobotState.EXPLORING
        print("✅ Wandering complete, resuming A* exploration")

def ai_exploration():
    """Main AI exploration function with A* pathfinding"""
    global is_wandering, wander_duration, stuck_counter, exploration_attempts
    global robot_state, path, current_path_visual
    
    update_zone_exploration()
    
    if is_exploration_complete():
        if exploring:
            print("🎉 All zones exploration complete!")
        return
    
    last_positions.append(robot_pos[:])
    
    if is_wandering:
        smart_wander()
        return
    
    # If no path or path is complete, find new target using A*
    if not path:
        path_points, target_zone = find_best_unexplored_target()
        
        if path_points:
            path.extend(path_points)
            current_path_visual = path_points[:]  # Store for visualization
            print(f"🎯 New A* path created: {len(path_points)} waypoints to zone {target_zone}")
        else:
            # No A* path found, start wandering
            print("🔄 No A* path found, starting wandering")
            start_wandering()

def update_traveled_path():
    """Update the traveled path"""
    if not traveled_path or (traveled_path[-1][0] != int(robot_pos[0]) or traveled_path[-1][1] != int(robot_pos[1])):
        traveled_path.append((int(robot_pos[0]), int(robot_pos[1])))

def get_exploration_percentage():
    """Calculate exploration percentage"""
    if total_explorable_cells == 0:
        return 0
    explored_count = len([c for c in explored_cells if c not in wall_cells])
    return (explored_count / total_explorable_cells) * 100

def draw_zones():
    """Draw zones with enhanced visualization"""
    for zone, zone_data in zones.items():
        if zone_data["explorable_cells"] == 0:
            continue
            
        min_x, min_y, max_x, max_y = zone_data["bounds"]
        
        if zone_data["completed"]:
            color = COMPLETED_ZONE_COLOR
            alpha = 40
        elif zone == current_zone:
            color = CURRENT_ZONE_COLOR
            alpha = 60
        else:
            color = ZONE_BORDER_COLOR
            alpha = 25
        
        zone_surface = pygame.Surface((max_x - min_x, max_y - min_y))
        zone_surface.set_alpha(alpha)
        zone_surface.fill(color)
        screen.blit(zone_surface, (min_x, min_y))
        
        pygame.draw.rect(screen, color, (min_x, min_y, max_x - min_x, max_y - min_y), 2)
        
        if zone_data["explorable_cells"] > 0:
            exploration_pct = (zone_data["explored_cells"] / zone_data["explorable_cells"]) * 100
            if exploration_pct > 0:
                text_x = min_x + 5
                text_y = min_y + 5
                status_icon = "✅" if zone_data["completed"] else "🔄" if zone == current_zone else ""
                draw_text(f"{status_icon}{exploration_pct:.0f}%", (text_x, text_y), color, small_font)

def draw_grid():
    """Draw exploration grid"""
    for cell in explored_cells:
        if cell not in wall_cells:
            world_pos = get_world_pos(cell)
            pygame.draw.rect(screen, EXPLORED_COLOR, 
                           (world_pos[0] - GRID_SIZE//2, world_pos[1] - GRID_SIZE//2, 
                            GRID_SIZE, GRID_SIZE))

def draw_walls():
    """Draw wall cells and actual walls"""
    for wall in walls:
        pygame.draw.rect(screen, WALL_COLOR, wall)
    
    for cell in wall_cells:
        world_pos = get_world_pos(cell)
        pygame.draw.rect(screen, (80, 80, 80),
                         (world_pos[0] - GRID_SIZE//2, world_pos[1] - GRID_SIZE//2,
                          GRID_SIZE, GRID_SIZE))

def draw_astar_path():
    """Draw the current A* path"""
    if len(current_path_visual) > 1:
        for i in range(len(current_path_visual) - 1):
            pygame.draw.line(screen, PATH_COLOR, 
                           current_path_visual[i], current_path_visual[i + 1], 3)
        
        # Draw waypoints
        for point in current_path_visual:
            pygame.draw.circle(screen, PATH_COLOR, (int(point[0]), int(point[1])), 4)

def draw_path_history():
    """Draw traveled path with gradient effect"""
    if len(traveled_path) > 1:
        for i in range(len(traveled_path) - 1):
            age_factor = i / len(traveled_path)
            red_intensity = int(80 + (175 * age_factor))
            color = (red_intensity, 40, 40)
            line_width = max(1, int(1 + (3 * age_factor)))
            pygame.draw.line(screen, color, traveled_path[i], traveled_path[i + 1], line_width)

        pygame.draw.circle(screen, (0, 255, 0), traveled_path[0], 10)  # Start
        pygame.draw.circle(screen, (255, 255, 0), traveled_path[-1], 8)  # Current

def draw_robot():
    """Draw robot with enhanced visualization"""
    color = WANDER_COLOR if robot_state == RobotState.WANDERING else ROBOT_COLOR
    
    pygame.draw.circle(screen, color, (int(robot_pos[0]), int(robot_pos[1])), ROBOT_RADIUS)
    pygame.draw.circle(screen, WHITE, (int(robot_pos[0]), int(robot_pos[1])), ROBOT_RADIUS, 2)

    # Direction indicator
    front_x = robot_pos[0] + (ROBOT_RADIUS + 5) * math.cos(math.radians(robot_angle))
    front_y = robot_pos[1] + (ROBOT_RADIUS + 5) * math.sin(math.radians(robot_angle))
    pygame.draw.line(screen, WHITE, robot_pos, (front_x, front_y), 3)

    # FOV visualization
    fov_start_angle = math.radians(robot_angle - FOV_ANGLE/2)
    fov_end_angle = math.radians(robot_angle + FOV_ANGLE/2)
    end_x1 = robot_pos[0] + FOV_RADIUS * math.cos(fov_start_angle)
    end_y1 = robot_pos[1] + FOV_RADIUS * math.sin(fov_start_angle)
    end_x2 = robot_pos[0] + FOV_RADIUS * math.cos(fov_end_angle)
    end_y2 = robot_pos[1] + FOV_RADIUS * math.sin(fov_end_angle)

    pygame.draw.line(screen, (0, 200, 0), robot_pos, (end_x1, end_y1), 1)
    pygame.draw.line(screen, (0, 200, 0), robot_pos, (end_x2, end_y2), 1)

# Initialize
calculate_total_explorable_cells()
scan_area(robot_pos, robot_angle)

print(f"🚀 Robot Zone Scanner with A* initialized!")
print(f"📊 Total zones: {len(zones)}")
print(f"📊 Total explorable cells: {total_explorable_cells}")

debug_counter = 0

# Main game loop
while True:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_SPACE:
                exploring = not exploring
                if exploring:
                    print("🚀 Starting A* zone scanning!")
                    is_wandering = False
                    stuck_counter = 0
                    wander_duration = 0
                    exploration_attempts = 0
                    last_positions.clear()
                    current_path_visual.clear()
                else:
                    print("⏸️ A* zone scanning stopped")
                    path.clear()
                    current_path_visual.clear()
                    is_wandering = False
            elif event.key == pygame.K_LEFT and not exploring:
                robot_angle = (robot_angle + 15) % 360
            elif event.key == pygame.K_RIGHT and not exploring:
                robot_angle = (robot_angle - 15) % 360
            elif event.key == pygame.K_r:
                explored_cells.clear()
                wall_cells.clear()
                traveled_path.clear()
                path.clear()
                current_path_visual.clear()
                stuck_counter = 0
                is_wandering = False
                wander_duration = 0
                exploration_attempts = 0
                last_positions.clear()
                robot_pos = [100, 50]
                for zone in zones:
                    zones[zone]["completed"] = False
                    zones[zone]["explored_cells"] = 0
                calculate_total_explorable_cells()
                print("🔄 Reset A* zone scanner!")
            elif event.key == pygame.K_c:
                traveled_path.clear()
                print("🧹 Cleared travel path history!")

    screen.fill(UNEXPLORED_COLOR)

    draw_zones()
    draw_grid()
    draw_walls()
    draw_astar_path()  # Draw A* path before travel history
    draw_path_history()
    draw_robot()

    if exploring:
        debug_counter += 1
        if debug_counter % 60 == 0:
            explored_count = len([c for c in explored_cells if c not in wall_cells])
            print(f"DEBUG: A* - Explored {explored_count}/{total_explorable_cells} cells")
            print(f"DEBUG: A* path length: {len(current_path_visual)}")
            print(f"DEBUG: Current zone: {current_zone}")
        
        ai_exploration()
        update_traveled_path()

    scan_area(robot_pos, robot_angle)

    if exploring and not is_wandering:
        move_robot()

    # UI
    draw_text("SPACE = Start/Stop | ← → = Rotate | R = Reset | C = Clear Path", (10, 10))

    exploration_pct = get_exploration_percentage()
    draw_text(f"Total Explored: {exploration_pct:.1f}% 🗺️", (10, 35))
    draw_text(f"Scanned Cells: {len([c for c in explored_cells if c not in wall_cells])}/{total_explorable_cells}", (10, 60))

    status = "A* Scanning..." if exploring and not is_wandering else "Smart Wandering..." if is_wandering else "Stopped"
    if is_exploration_complete() and exploring:
        status = "✅ A* ZONE SCAN COMPLETE!"
    draw_text(f"Status: {status}", (10, 85))

    if traveled_path:
        total_distance = 0
        for i in range(len(traveled_path) - 1):
            total_distance += math.hypot(
                traveled_path[i+1][0] - traveled_path[i][0],
                traveled_path[i+1][1] - traveled_path[i][1]
            )
        draw_text(f"Travel Distance: {int(total_distance)} px 📏", (10, 110))
        draw_text(f"Path Points: {len(traveled_path)} 📍", (10, 135))

    if is_wandering:
        wander_progress = min(100, (wander_duration / max_wander_time) * 100)
        draw_text(f"🔄 Smart Wander Mode ({wander_progress:.0f}%)", (10, 160))

    # A* specific debug info
    if exploring:
        draw_text(f"A* Path: {len(current_path_visual)} waypoints", (10, 185), (255, 0, 255))
        draw_text(f"Queue: {len(path)} moves", (10, 210), (255, 0, 255))

    pygame.display.flip()
    clock.tick(60)

pygame 2.6.1 (SDL 2.28.4, Python 3.10.16)
Hello from the pygame community. https://www.pygame.org/contribute.html
🗺️ Created 70 zones
📊 Total explorable cells: 2265
🚀 Robot Zone Scanner with A* initialized!
📊 Total zones: 70
📊 Total explorable cells: 2265
🚀 Starting A* zone scanning!
🎯 Entered zone (0, 0)
🎯 A* path to unexplored cell in current zone (0, 0)
🎯 New A* path created: 2 waypoints to zone (0, 0)
🎯 A* path to unexplored cell in current zone (0, 0)
🎯 New A* path created: 3 waypoints to zone (0, 0)
🎯 A* path to unexplored cell in current zone (0, 0)
🎯 New A* path created: 5 waypoints to zone (0, 0)
🎯 A* path to unexplored cell in current zone (0, 0)
🎯 New A* path created: 4 waypoints to zone (0, 0)
DEBUG: A* - Explored 44/2265 cells
DEBUG: A* path length: 4
DEBUG: Current zone: (0, 0)
🎯 A* path to unexplored cell in current zone (0, 0)
🎯 New A* path created: 4 waypoints to zone (0, 0)
✅ Zone (0, 0) completed! (36/36 cells - 100%)
🚀 A* path to nearest incomplete zone (0, 1) (dist

SystemExit: 

c:\Users\earth\anaconda3\envs\Car\lib\site-packages\IPython\core\interactiveshell.py:3587: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
